In [22]:
import networkx as nx
from amplpy import AMPL
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
import sys
import os
import contextlib
from sklearn.decomposition import PCA
import random

import matplotlib.lines as mlines
import matplotlib.patches as mpatches

import warnings
warnings.filterwarnings("ignore")

import multiprocessing
# !pip install pathos

# from pathos.multiprocessing import ProcessingPool as Pool

class WaterNetworkOptimizer:
    def __init__(self, model_file, data_file, data_number, verbose=True):
        self.ampl = AMPL()
        self.model_file = model_file
        self.data_file = data_file
        self.data_number = data_number
        self.total_cost = None
        self.network_graph = None
        self.solve_result = None
        self.solver_time = 0
        self.best_acyclic_flow = None
        self.number_of_nlp = 0
        self.verbose = verbose

    def load_model(self):
        """Load the model and data."""
        self.ampl.reset()
        self.ampl.read(self.model_file)
        self.ampl.read_data(self.data_file)
        
        self.nodes = self.ampl.getSet('nodes')
        self.source = self.ampl.getSet('Source')
        self.arcs = self.ampl.getSet('arcs')
        self.pipes = self.ampl.getSet('pipes')
        
        self.L = self.ampl.getParameter('L').to_dict()
        self.D = self.ampl.getParameter('D').to_dict()
        self.C = self.ampl.getParameter('C').to_dict()
        self.P = self.ampl.getParameter('P').to_dict()
        self.R = self.ampl.getParameter('R').to_dict()
        self.E = self.ampl.getParameter('E').to_dict()
        self.d = self.ampl.getParameter('d').to_dict()

        self.delta = 0.1
        self.p = 1.852
        self.omega = 10.67

    def create_digraph(self):
        nodes_list = [i for i in self.ampl.getSet('nodes')]
        edges_list = self.ampl.getSet('arcs').to_list()
        self.network_graph = nx.DiGraph()
        self.network_graph.add_nodes_from(nodes_list)
        self.network_graph.add_edges_from(edges_list)
        print(nodes_list)
        print(edges_list)

    def display_results(self):
        """Display relevant results from the optimization."""
        self.ampl.eval("display {(i,j) in arcs, k in pipes:l[i,j,k]>1} l[i,j,k];")
        self.ampl.eval("display {(i,j) in arcs}: q[i,j];")
        self.ampl.eval("display h;")
        self.ampl.eval("display solve_result;")
        self.total_cost = self.ampl.get_objective("total_cost").value()
        print("Objective:", self.total_cost)
        # self.ampl.eval("display {(i,j) in arcs} h[i] - h[j];")
        # self.ampl.eval("display {i in nodes} h[i] - (E[i] + P[i]);")

    # def plot_graph(self):
    #     print("Edges of the graph:",self.network_graph.edges())
    #     plt.figure(figsize=(10, 8))
    #     nx.draw_spectral(self.network_graph, with_labels=True)
    #     plt.show()


    def plot_graph1(self, super_source_out_arc=None, best_arc=None,current_cost = None, iteration= 1):
        # print("Edges of the graph:", self.network_graph.edges())
        indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
        
        plt.figure(figsize=(10, 8))
        pos = nx.spectral_layout(self.network_graph)
        nx.draw_networkx_nodes(self.network_graph, pos, node_color='lightblue', node_size=200)
        
        if indegree_2_or_more:
            nx.draw_networkx_nodes(self.network_graph, pos, nodelist=indegree_2_or_more, node_color='orange', node_size=200)
            
        nx.draw_networkx_labels(self.network_graph, pos)
        nx.draw_networkx_edges(self.network_graph, pos, edge_color='black')
        
        if super_source_out_arc:
            nx.draw_networkx_edges(self.network_graph, pos, edgelist=super_source_out_arc, edge_color='red', width=1)
            
        if best_arc:
            nx.draw_networkx_edges(self.network_graph, pos, edgelist=[best_arc], edge_color='magenta', width=1)
            
        plt.title(f"total_cost:{self.total_cost}")
        plt.savefig(f"/home/nitishdumoliya/waterNetwork/model/figure/d{self.data_number}_iteration_{iteration}.png")
        plt.box(False)
        plt.show()

    def plot_graph(self, super_source_out_arc=None, current_cost = None, iteration = 1, edge_weights= None, h = None, D = None, arc=(0,0), l ={}, C = {}):
        # self.network_graph = nx.DiGraph()
        # self.network_graph.add_edges_from(edges)
        # Node positions as per your specifications
        
        new_positions0 = {
             2 : (2000.00, 3000.00),
             3 : (1000.00, 3000.00),        
             4 : (2000.00, 2000.00),        
             5 : (1000.00, 2000.00),       
             6 : (2000.00, 1000.00),       
             7 : (1000.00, 1000.00),      
             1 : (3000.00, 3000.00)     
            }  
        
        new_positions1 = {
            2: (6000.00, 2000.00),
            3: (6000.00, 4000.00),
            4: (7500.00, 4000.00),
            5: (9000.00, 4000.00),
            6: (10000.00, 4000.00),
            7: (10000.00, 6000.00),
            8: (10000.00, 8000.00),
            9: (10000.00, 10000.00),
            10: (9000.00, 10000.00),
            11: (9000.00, 11500.00),
            12: (9000.00, 13000.00),
            13: (7000.00, 13000.00),
            14: (8000.00, 10000.00),
            15: (7000.00, 10000.00),
            16: (6000.00, 10000.00),
            17: (6000.00, 8500.00),
            18: (6000.00, 7000.00),
            19: (6000.00, 5500.00),
            20: (4500.00, 4000.00),
            21: (4500.00, 2000.00),
            22: (4500.00, 0.00),
            23: (3000.00, 4000.00),
            24: (3000.00, 7000.00),
            25: (3000.00, 10000.00),
            26: (4000.00, 10000.00),
            27: (5000.00, 10000.00),
            28: (1500.00, 4000.00),
            29: (0.00, 4000.00),
            30: (0.00, 7000.00),
            31: (0.00, 10000.00),
            32: (1500.00, 10000.00),
            1: (8000.00, 0.00)
        }       
        
        new_positions16 = {
            2: (5679.61, 9538.83),
            3: (4862.46, 9538.83),
            4: (2750.81, 9474.11),
            5: (1852.75, 8357.61),
            6: (1974.11, 6076.05),
            7: (1974.11, 5149.68),
            8: (4235.44, 5076.86),
            9: (6411.81, 5093.04),
            10: (5412.62, 7888.35),
            11: (4510.52, 8264.56),
            12: (3033.98, 9243.53),
            13: (2301.78, 8078.48),
            14: (2944.98, 7669.90),
            15: (3786.41, 7139.97),
            16: (4830.10, 6480.58),
            17: (7099.51, 8438.51),
            18: (5505.66, 8450.65),
            19: (3563.92, 8839.00),
            20: (3167.48, 7532.36),
            21: (2730.58, 7285.60),
            22: (3511.33, 6666.67),
            23: (4097.90, 6286.41),
            24: (3337.38, 5121.36),
            25: (4530.74, 6011.33),
            26: (4215.21, 7783.17),
            27: (5194.17, 7055.02),
            28: (5218.45, 5089.00),
            29: (5622.98, 5999.19),
            30: (5950.65, 5796.93),
            31: (6614.08, 7621.36),
            32: (5380.26, 7544.50),
            33: (6318.77, 7281.55),
            34: (6549.35, 7212.78),
            35: (6585.76, 6092.23),
            36: (7152.10, 6104.37),
            1: (7111.65, 7532.36),
            37: (7669.90, 7783.17)
        }

        # # Update node positions
        pos = new_positions0
        # pos = nx.spectral_layout(self.network_graph)

        cost = {}

        for (i,j) in self.ampl.getSet('arcs'):
            cost[i,j] = sum(l[i,j,k] * C[k] for k in self.ampl.getSet('pipes'))
        
        plt.figure(figsize=(10, 8))
        cmap = plt.cm.plasma

        nx.draw_networkx_nodes(self.network_graph, pos, node_color='lightblue', node_size=300, label='Regular Nodes')

        indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
        if indegree_2_or_more:
            nx.draw_networkx_nodes(self.network_graph, pos, nodelist=indegree_2_or_more, node_color='orange', node_size=300, label='Nodes with In-Degree ≥ 2')

        nx.draw_networkx_labels(self.network_graph, pos, font_size=10)

        nx.draw_networkx_edges(self.network_graph, pos, arrowstyle="->", arrowsize=12, edge_color='gray', label='Regular Arcs')

        if super_source_out_arc:
            nx.draw_networkx_edges(self.network_graph, pos, edgelist=super_source_out_arc,arrowstyle="->", arrowsize=12, edge_color='red', width=1, label='Fix arc direction')

        # if best_arc:
        #     nx.draw_networkx_edges(self.network_graph, pos, edgelist=[best_arc],arrowstyle="->", arrowsize=12, edge_color='magenta', width=1, label = 'Best arc')
        # Annotate node demands
        if h:
            for node, (x, y) in pos.items():
                demand = h.get(node, 0)  # Get the head for the node, default to 0 if not in dictionary
                plt.text(x, y + 250, f"{demand:.2f}", fontsize=8, color='blue', ha='center')  # Annotate demand below the node
                # plt.text(x, y + 100, f"{demand:.2f}", fontsize=8, color='blue', ha='center')  # Annotate demand below the node
                # plt.text(x, y -100 , f"{demand:.2f}", fontsize=8, color='magenta', ha='center')  # Annotate demand below the node
                # plt.text(mid_x, mid_y + 50 , f"{weight:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
                # plt.text(mid_x, mid_y + 100, f"{value:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        if D:
            for node, (x, y) in pos.items():
                demand = D.get(node, 0)  # Get the demand for the node, default to 0 if not in dictionary
                plt.text(x, y - 300, f"{demand:.2f}", fontsize=8, color='magenta', ha='center')  # Annotate demand below the node
                # plt.text(x, y -100 , f"{demand:.2f}", fontsize=8, color='magenta', ha='center')  # Annotate demand below the node
            

        if edge_weights:
            for (u, v), weight in edge_weights.items():
                # if self.network_graph.has_edge(u, v):
                mid_x = (pos[u][0] + pos[v][0]) / 2  # Midpoint x-coordinate
                mid_y = (pos[u][1] + pos[v][1]) / 2  # Midpoint y-coordinate
                # plt.text(mid_x, mid_y+100 , f"{weight:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
                plt.text(mid_x, mid_y + 50 , f"{weight:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        if cost:
            for (u, v), value in cost.items():
                # if self.network_graph.has_edge(u, v):
                mid_x = (pos[u][0] + pos[v][0]) / 2  # Midpoint x-coordinate
                mid_y = (pos[u][1] + pos[v][1]) / 2  # Midpoint y-coordinate
                # plt.text(mid_x, mid_y-300 , f"{round(value)}", fontsize=8, color='purple', ha='center')  # Annotate weight on edge
                # plt.text(mid_x, mid_y + 100, f"{value:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        pipe_dia_arc = {}
        for (i,j) in self.arcs:
            list_=[]
            for k in self.pipes:
                if l[i,j,k]>= 1e-5:
                    list_.append(k)
            pipe_dia_arc[i,j] = list_
        
        if pipe_dia_arc:
            for (u, v), weight in pipe_dia_arc.items():
                # if self.network_graph.has_edge(u, v):
                mid_x = (pos[u][0] + pos[v][0]) / 2  # Midpoint x-coordinate
                mid_y = (pos[u][1] + pos[v][1]) / 2  # Midpoint y-coordinate
                # plt.text(mid_x, mid_y+550 , f"{weight}", fontsize=8, color='purple', ha='center')  # Annotate weight on edge
                plt.text(mid_x, mid_y+150 , f"{weight}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        
        regular_node_patch = mpatches.Patch(color='lightblue', label='Regular Nodes')
        indegree_node_patch = mpatches.Patch(color='orange', label='Nodes with In-Degree ≥ 2')
        regular_edge_line = mlines.Line2D([], [], color='black', label='Regular Arcs')
        super_source_edge_line = mlines.Line2D([], [], color='red', label='Fix arc direction')
        # best_edge_line = mlines.Line2D([], [], color='magenta', label='Best Arc')
        plt.legend(handles=[regular_node_patch, indegree_node_patch, regular_edge_line, super_source_edge_line], loc='lower right')
        
        cost = round(self.total_cost)
        # res = f"{cost:,}"
        plt.title(f"total_cost:{self.format_indian_number(cost)}")
        # (u,v) = arc
        # plt.savefig(f"/home/nitishdumoliya/waterNetwork/model/figure/d{self.data_number}_iteration_{iteration}.png")
        plt.box(True)
        plt.show()

    
    def cycle_basis(self):
        root = self.ampl.getSet('Source').to_list()[0]
        nodes_list = [i for i in self.ampl.getSet('nodes')]
        edges_list = self.ampl.getSet('arcs').to_list() 
        uwg = nx.Graph()
        uwg.add_nodes_from(nodes_list)
        uwg.add_edges_from(edges_list)
        # print("Edges in the undirected graph:", edges_list)
        print("cycle basis for given water network: ",nx.cycle_basis(uwg, root))
        
    def generate_random_acyclic_from_solution(self, q):
        # print("Generate the acyclic network using ipopt solution")
        
        self.network_graph = nx.DiGraph()
        self.network_graph.add_nodes_from(self.nodes)
        
        # q = self.ampl.getVariable('q').getValues().toDict()
        for (i,j) in self.arcs:
            if q[i,j] >= 0:
                self.network_graph.add_edge(i,j)
            else:
                self.network_graph.add_edge(j,i)
        
        return self.network_graph

    
    def generate_random_acyclic_graph(self):
        uwg = nx.Graph()
        nodes_list = [i for i in self.ampl.getSet('nodes')]
        edges_list = self.ampl.getSet('arcs').to_list()
        uwg.add_nodes_from(nodes_list)
        uwg.add_edges_from(edges_list)
        print("Edges in the undirected graph:", edges_list)
        
        # Generate a random spanning tree using Wilson's algorithm
        random_tree = nx.random_spanning_tree(uwg)
        
        # Retrieve the root from the AMPL source set
        root_l = self.ampl.getSet('Source').to_list()
        root = root_l[0]
        print("Root node:", root)

        # Ensure the root is present in the random tree
        if root not in random_tree.nodes:
            raise ValueError("The specified root must be a node in the graph.")

        # Create a directed graph from the random tree starting from the specified root
        self.network_graph = nx.DiGraph()
        visited = set()

        def dfs(node):
            visited.add(node)
            for neighbor in random_tree.neighbors(node):
                if neighbor not in visited:
                    self.network_graph.add_edge(node, neighbor) 
                    dfs(neighbor)

        # Start DFS from the specified root
        dfs(root)

        # Draw the initial directed tree
        plt.figure(figsize=(15, 10))
        plt.subplot(121)
        nx.draw_spectral(self.network_graph, with_labels=True, node_color='lightgreen', font_weight='bold', arrows=True)
        plt.title("Directed Spanning Tree")

        # Add remaining edges from the original graph and check for cycles
        for u, v in uwg.edges():
            if not self.network_graph.has_edge(u, v):  
                self.network_graph.add_edge(u, v)  
                if not nx.is_directed_acyclic_graph(self.network_graph):  
                    self.network_graph.remove_edge(u, v)  
                    self.network_graph.add_edge(v, u)  

        # Draw the final directed graph after adding remaining edges
        plt.subplot(122)
        nx.draw_spectral(self.network_graph, with_labels=True, node_color='lightgreen', font_weight='bold', arrows=True)
        plt.title("Acyclic Directed Graph")
        plt.show()

    def update_model(self):
        # print("Fix the arcs direction using the acyclic network\n")
        edges_list = [(arc[0],arc[1]) for arc in self.ampl.getSet('arcs')]
        for edge in self.network_graph.edges:
            i, j = edge
            if edge in edges_list:
                self.ampl.eval(f"s.t. flow_direction{i}_{j}: q[{i},{j}] >=0;")
                # self.ampl.eval(f"s.t. head_bound_left{i}_{j}: E[{j}]+P[{j}] <= h[{j}];")
                # self.ampl.eval(f"s.t. head_bound_right{i}_{j}: E[{j}] + P[{j}] <= h[{i}];")
            else:
                self.ampl.eval(f"s.t. flow_direction{i}_{j}: q[{j},{i}] <=0;")
                # self.ampl.eval(f"s.t. head_bound_left{i}_{j}: E[{i}]+P[{i}] <= h[{i}];")
                # self.ampl.eval(f"s.t. head_bound_right{i}_{j}: E[{i}] + P[{i}] <= h[{j}];")  


    def is_valid_edge(self, source, target):
        """Check if adding the directed edge (source -> target) maintains acyclicity."""
        self.network_graph.add_edge(source, target)  # Temporarily add the edge
        is_dag = nx.is_directed_acyclic_graph(self.network_graph)  # Check for acyclicity
        self.network_graph.remove_edge(source, target)  # Remove the edge after checking
        return is_dag  # Return True if it maintains acyclicity     
    
    def check_incoming_arcs(self):
        root = list(self.ampl.getSet('Source'))[0]
        # Iterate over all nodes in the graph
        for node in self.network_graph.nodes():
            # Skip the root node
            if node == root:
                continue
            # Check if the in-degree of the node is at least 1
            if self.network_graph.in_degree(node) < 1:
                # print(f"Node {node} does not have any incoming arcs.")
                return False
        # print("All nodes except the root have at least one incoming arc.")
        return True
            
    def format_indian_number(self,num):
        num_str = str(num)
        if len(num_str) <= 3:
            return num_str
        else:
            # Split the number into the last three digits and the rest
            last_three = num_str[-3:]
            remaining = num_str[:-3]
            # Add commas every two digits in the remaining part
            remaining = ','.join([remaining[max(i - 2, 0):i] for i in range(len(remaining), 0, -2)][::-1])
            return remaining + ',' + last_three

    def fix_leaf_arc_flow(self):
        graph = nx.Graph()
        arc_set = self.ampl.getSet('arcs').to_list()  
        graph.add_edges_from(arc_set)
        D = self.ampl.getParameter('D').getValues().to_dict()  
        source = self.ampl.getSet('Source').to_list()[0]
        fixed_arcs = set()
        # print("\nPresolve the model for fixing the flow value in the leaf arcs")
        # print("Source:",self.ampl.getSet('Source').to_list())

        while True:
            leaf_nodes = [node for node in graph.nodes if graph.degree[node] == 1]
            # print("leaf_nodes:", leaf_nodes)
            if not leaf_nodes:  
                break

            for leaf in leaf_nodes:
                neighbor = next(graph.neighbors(leaf))
                if (neighbor, leaf) in arc_set:
                    edge = (neighbor, leaf)
                    if edge not in fixed_arcs:  
                        if leaf == source:
                            flow_value = D[leaf]
                            D[neighbor] = (D[neighbor]+flow_value)
                            source = neighbor
                        else:
                            flow_value = D[leaf]
                            D[neighbor] = D[neighbor] + flow_value
                        self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        # print(f"Fixing flow for arc {edge}: {flow_value}")
                        fixed_arcs.add(edge)  

                    graph.remove_node(leaf)
                else:
                    edge = (leaf, neighbor)
                    if edge not in fixed_arcs:  
                        if leaf == source:
                            flow_value = -D[leaf]
                            D[neighbor] = D[neighbor]-flow_value
                            source = neighbor
                            self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        elif neighbor == source:
                            flow_value = -D[leaf]
                            D[neighbor] = D[neighbor] - D[leaf] 
                            self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        else:
                            flow_value = -D[leaf]
                            D[neighbor] += -flow_value
                            self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        # print(f"Fixing flow for arc {edge}: {flow_value}")
                        fixed_arcs.add(edge)  
                    graph.remove_node(leaf)
        # print("All leaf arc flows have been fixed.")


    def is_cycle(self, graph, start_node, end_node, visited_copy, parent):
        visited_copy[start_node] = True
        # print(f"Is node {end_node} in cycle?")
        for neighbor in graph.neighbors(start_node):
            # print("visited",neighbor,visited_copy[neighbor])
            if not visited_copy[neighbor]:
                # print("neighbor of node", start_node, "is", neighbor)
                isCycle = self.is_cycle(graph, neighbor, end_node, visited_copy, start_node)
                if isCycle:
                    return True
            else:
                # print("parent:", parent)
                if parent != neighbor:
                    if end_node == neighbor:
                        # print(f"Node {end_node} is in cycle")
                        return True
        return False

    def presolve(self, graph, node, visited, parent, set_arc):
        visited_copy = visited.copy()
        # print(visited_copy)
        isCycle = self.is_cycle(graph, node, node, visited_copy, parent)
        # print(f"Is node {node} in cycle?",isCycle)
        visited[node] = True
        if isCycle:
            for neighbor in graph.neighbors(node):
                if parent!=neighbor:
                    set_arc.append((node,neighbor))
                    # print("Fix the arc", (node, neighbor))
            return set_arc
        else:
            for neighbor in graph.neighbors(node):
                if parent != neighbor:
                    set_arc.append((node,neighbor))
                    # print(set_arc)
                    # print("Fix the arc", (node, neighbor))
                    # print("neighbor:", neighbor)
                    self.presolve(graph, neighbor, visited, node, set_arc)
        return set_arc

    def fix_arc_set(self):
        graph = nx.Graph()
        arc_set = self.ampl.getSet('arcs').to_list()
        graph.add_edges_from(arc_set)
        visited = {node: False for node in graph.nodes()}
        source = self.ampl.getSet('Source').to_list()[0]
        set_arc = []
        # print("\nPresolve the model for fixing the arc direction")
        set_ = self.presolve(graph, source, visited, -1, set_arc)
        # print("fixed arc direction:",set_, "\n") 
        return set_


    def update_initial_points(self,l_solution, q_solution, h_solution):
        for (i, j, k), val in l_solution.items():
            self.ampl.eval(f'let l[{i},{j},{k}] := {val};')
        for (i, j), val in q_solution.items():
            self.ampl.eval(f'let q[{i},{j}] := {val};')
        for i, val in h_solution.items():
            self.ampl.eval(f'let h[{i}] := {val};')

    def update_initial_points1(self, l_solution, q_solution, h_solution, t_solution,all_duals, inarc):
        for (i, j, k), val in l_solution.items():
            self.ampl.eval(f'let l[{i},{j},{k}] := {val};')
        
        edge_list_network = self.network_graph.edges
        
        for i, val in h_solution.items():
            self.ampl.eval(f'let h[{i}] := {val};')
        
        for (i, j), val in q_solution.items():
            edge = (i, j) 
            if edge in edge_list_network:
                if (i,j) not in inarc:
                    # print(f"self.ampl.eval(let q[{i},{j}] := {val};)")
                    self.ampl.eval(f"let q[{i},{j}] := {val} ;")
                else:
                    # print(f"self.ampl.eval(let q[{i},{j}] := {-val};)")
                    self.ampl.eval(f"let q[{i},{j}] := {val} ;")
            else:
                if (j,i) not in inarc:
                    # print(f"self.ampl.eval(let q[{i},{j}] := {val};)")
                    self.ampl.eval(f"let q[{i},{j}] := {val};")
                else:
                    # print(f"self.ampl.eval(let q[{j},{i}] := {-val};)")
                    self.ampl.eval(f"let q[{j},{i}] := {val} ;")
        for i, val in t_solution.items():
            self.ampl.eval(f'let t[{i}] := {val};')
            
        current_duals = {}
        for con_name, val in self.ampl.get_constraints():
            dual_values = val.get_values()
            current_duals[con_name] = dual_values

        # Initialize dual values for all constraints
        # for con_name, dual_values in all_duals.items():
            # if con_name in current_duals:
                # Initialize dual values for each constraint
                # self.ampl.get_constraint(con_name).set_values(dual_values)
            # else:
            #     print(f"Skipping initialization for constraint: {con_name} (not in current model)")
    
    def update_initial_points_with_perturbation(self, l_solution, q_solution, h_solution,all_duals, inarc, delta=0.1):
        edge_list_network = self.network_graph.edges
        L = self.ampl.getParameter('L').getValues().to_dict()
        # Perturb l values
        for (i, j, k), val in l_solution.items():
            if (i,j) not in inarc:
                if val>= 1e-5:
                    perturbation = random.gauss(0, 1)
                    new_val = val + perturbation
                    # if val >= 1e-5:
                    self.ampl.eval(f'let l[{i},{j},{k}] := {new_val};')
                else:
                    self.ampl.eval(f'let l[{i},{j},{k}] := {0};')
            else:
                if val>= 1e-5:
                    perturbation = random.gauss(0, 1)
                    new_val = val + perturbation
                    self.ampl.eval(f'let l[{i},{j},{k}] := {new_val};')
                else:
                    self.ampl.eval(f'let l[{i},{j},{k}] := {0};')
                    
        # Perturb h values
        for i, val in h_solution.items():
            perturbation = random.gauss(0, 1)
            new_val = val + perturbation
            self.ampl.eval(f'let h[{i}] := {new_val};')

        # Modify q values based on heuristic
        for (i, j), val in q_solution.items():
            edge = (i, j)
            perturbation = random.gauss(0, 1)
            if edge in edge_list_network:
                if (i, j) not in inarc:
                    self.ampl.eval(f"let q[{i},{j}] := {val + perturbation};")
                else:
                    self.ampl.eval(f"let q[{i},{j}] := {(val + perturbation)};")
            else:
                if (j, i) not in inarc:
                    self.ampl.eval(f"let q[{i},{j}] := {val + perturbation};")
                else:
                    self.ampl.eval(f"let q[{j},{i}] := {(val + perturbation)};")
        
        current_duals = {}
        for con_name, val in self.ampl.get_constraints():
            dual_values = val.get_values()
            current_duals[con_name] = dual_values

        # Initialize dual values for all constraints
        for con_name, dual_values in all_duals.items():
            if con_name in current_duals:
                # Initialize dual values for each constraint
                self.ampl.get_constraint(con_name).set_values(dual_values)

    def acyclic_arcs(self):
        network_graph = self.best_acyclic_flow
        indegree_2_or_more = [node for node, indeg in network_graph.in_degree() if indeg >= 2]
        acyclic_arc = set()
        for node in indegree_2_or_more:
            # print("Node:", node,"in_degree:", self.network_graph.in_degree(node))
            for edge in list(network_graph.in_edges(node)):
                (u, v) = edge
                if (u,v) not in self.super_source_out_arc :
                    network_graph.remove_edge(u,v)
                    network_graph.add_edge(v,u)
                    acy_check = nx.is_directed_acyclic_graph(network_graph)
                    in_arc_check = self.check_incoming_arcs()
                    # print("Acyclic", acy_check and in_arc_check)
                    if acy_check and in_arc_check:
                        acyclic_arc.add((u,v))

                    network_graph.remove_edge(v, u)
                    network_graph.add_edge(u, v) 
        return acyclic_arc


    def process_arc(self, arc):
        u, v = arc
        # Reverse the arc
        self.network_graph.remove_edge(u, v)
        self.network_graph.add_edge(v, u)
        acy_check = nx.is_directed_acyclic_graph(self.network_graph)
        in_arc_check = self.check_incoming_arcs()

        if acy_check and in_arc_check:
            self.load_model()
            self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
            self.fix_leaf_arc_flow()
            self.update_initial_points1(self.l, self.q, self.h, self.t, self.all_duals, self.inarc)

            if (u, v) in self.arcs:
                self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{u}, {v}]<=0;")
            else:
                self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{v}, {u}]>=0;")

            self.solve()

            if self.solve_result == "solved":
                l = self.ampl.getVariable('l').getValues().to_dict()
                q = self.ampl.getVariable('q').getValues().to_dict()
                h = self.ampl.getVariable('h').getValues().to_dict()
                t = self.ampl.getVariable('t').getValues().to_dict()

                return {
                    "arc": arc,
                    "acyclic": True,
                    "total_cost": self.total_cost,
                    "solve_result": self.solve_result,
                    "solve_time": round(self.ampl.get_value('_solve_elapsed_time'), 2),
                    "improved": self.total_cost < self.current_cost,
                    "l": l,
                    "q": q,
                    "h": h,
                    "t": t,
                    "acyclic_flow": self.network_graph.copy()
                }

        # Revert the arc if not successful
        self.network_graph.remove_edge(v, u)
        self.network_graph.add_edge(u, v)
        return {"arc": arc, "acyclic": False}

    def run_parallel1(self):
        improved = False
        best_result = None
        
        while True:
            # Use multiprocessing to process arcs in parallel
            with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
                results = pool.map(self.process_arc, self.acyclic_arc_set)

            # Filter valid solutions and find the best result
            valid_results = [r for r in results if r["acyclic"] and "total_cost" in r]
            if valid_results:
                best_result = min(valid_results, key=lambda r: r["total_cost"])

                if best_result["total_cost"] < self.current_cost:
                    self.current_cost = best_result["total_cost"]
                    self.best_acyclic_flow = best_result["acyclic_flow"]
                    self.l = best_result["l"]
                    self.q = best_result["q"]
                    self.h = best_result["h"]
                    self.t = best_result["t"]
                    improved = True
                    
                    print(f"New best solution found! Total cost: {self.current_cost}")
                else:
                    improved = False
            else:
                print("No improved solutions found in this iteration.")
            
            # If no improvement, stop the loop
            if not improved:
                break

            # Update the graph and prepare for the next iteration
            self.network_graph = self.best_acyclic_flow.copy()
            self.acyclic_arc_set = self.acyclic_arcs()  # Recompute the arcs to process
        
        # Return the best result
        return best_result, improved
    
    def iterate_arc(self, iteration, improved, current_cost, best_arc):
        improved = False
        self.network_graph = self.best_acyclic_flow.copy()
        
        # print("Acyclic network arcs direction: ",self.network_graph.edges())
        # print("Fixed arc set:",self.super_source_out_arc)
        
        BEST_ARC = []
        # BEST_ARC.append(best_arc)
        
        self.inarc = []
        for node in self.indegree_2_or_more:
            for (u, v) in list(self.network_graph.in_edges(node)):
                if (u, v) in self.arcs:
                    self.inarc.append((u,v))
                else:
                    self.inarc.append((v,u))
        print("inarc:", self.inarc)
        inarc_ = self.inarc
        
        inarc_set = []
        for (i, j) in self.inarc:
            if (i, j) in self.arcs:
                inarc_set.append(f"({i},{j})")
            else:
                inarc_set.append(f"({j},{i})")
        
        # Convert the list into a string compatible with AMPL
        inarc_set = ", ".join(inarc_set)
        
        # for node in self.indegree_2_or_more:
        #     print("Node:", node,"in_degree:", self.network_graph.in_degree(node))
        #     for u,v in list(self.network_graph.in_edges(node)):
        for (u,v) in self.acyclic_arc_set:
            self.network_graph.remove_edge(u,v)
            self.network_graph.add_edge(v,u)
            acy_check = nx.is_directed_acyclic_graph(self.network_graph)
            in_arc_check = self.check_incoming_arcs()
            # print("Acyclic", acy_check and in_arc_check)
            if acy_check and in_arc_check:
                #l_sol, q_sol, h_sol = self.generate_initial_points()
                self.load_model()
                # self.ampl.eval(f"set inarc := {{{inarc_set}}};")
                # self.ampl.eval(f"set indegree_node := {{{set(self.indegree_2_or_more)}}};")
                self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
                self.fix_leaf_arc_flow()
                self.update_initial_points1(self.l, self.q, self.h, self.t, self.all_duals, self.inarc)
                # self.update_model()
                self.ampl.eval(f"param Q_max = sum{{k in nodes diff Source }} D[k];")   
                # self.ampl.eval("subject to con7{(i,j) in arcs}: -sum{k in nodes diff Source} D[k] <= q[i,j];")
                # self.ampl.eval("subject to con8{(i,j) in arcs}: q[i,j] <= sum{k in nodes diff Source} D[k];")
                
                if (u,v) in self.arcs:
                    self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{u}, {v}]<=0;")
                    self.ampl.eval(f"s.t. flow_bound_left_{u}_{v}: -Q_max <= q[{u}, {v}];")
                else:
                    self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{v}, {u}]>=0;")
                    self.ampl.eval(f"s.t. flow_bound_right_{u}_{v}: q[{v}, {u}] <= Q_max;")
                self.solve()
                
                l = self.ampl.getVariable('l').getValues().to_dict()
                q = self.ampl.getVariable('q').getValues().to_dict()
                h = self.ampl.getVariable('h').getValues().to_dict()
                t = self.ampl.getVariable('t').getValues().to_dict()
                
                if self.solve_result == "solved":
                    
                    if self.total_cost < current_cost:
                        # print("Arc", (u,v),"Acyclic:", acy_check and in_arc_check, "Best optimal: ", '{:,}'.format(round(current_cost)), "New optimal: ", '{:,}'.format(round(self.total_cost)), "Solve_time:", self.ampl.get_value('_solve_elapsed_time'), "Solve_result: ", self.solve_result, "Improved: Yes")
                        
                        print(f"Arc: {(u,v)}",
                              f"Acyclic: {acy_check and in_arc_check}",
                              f"Best_optimal: {self.format_indian_number(round(current_cost))}", 
                              f"New_optimal: {self.format_indian_number(round(self.total_cost))}", 
                              f"Solve_time: {round(self.ampl.get_value('_solve_elapsed_time'), 2)}", 
                              f"Solve_result: {self.solve_result}", "Improved: Yes", 
                              f"Time_count: {round(time.time() - self.start_time, 2)}")

                        current_cost = self.total_cost
                        improved = True
                        self.network_graph = self.generate_random_acyclic_from_solution(q)
                        
                        self.best_acyclic_flow = self.network_graph.copy()
                        self.indegree_2_or_more = [node for node, indeg in self.best_acyclic_flow.in_degree() if indeg >= 2]
                        # print("indegree_2_or_more:", self.indegree_2_or_more)
                        
                        best_arc = (v,u)
                        self.l = l 
                        self.q = q
                        self.h = h 
                        self.t = t

                        self.all_duals = {}
                        for con_name, val in self.ampl.get_constraints():
                            # Get dual values for each constraint
                            dual_values = val.getValues()
                            self.all_duals[con_name] = dual_values

                        self.inarc = []
                        for node in self.indegree_2_or_more:
                            for (u, v) in self.best_acyclic_flow.in_edges(node):
                                if (u, v) in self.arcs:
                                    if (u,v) not in self.super_source_out_arc:
                                        self.inarc.append((u,v))
                                else:
                                    self.inarc.append((v,u))
                        # self.plot_graph(self.super_source_out_arc, current_cost, iteration, self.q, self.h, self.D, self.l, self.C)
                        self.acyclic_arc_set = self.acyclic_arcs()
                        
                        break
                    else:
                        # print("Arc", (u,v),"Acyclic:", acy_check and in_arc_check, "Best optimal: ", '{:,}'.format(round(current_cost)), "New optimal: ", '{:,}'.format(round(self.total_cost)), "Solve_time:", self.ampl.get_value('_solve_elapsed_time'), "Solve_result: ", self.solve_result, "Improved: No")

                        print(f"Arc: {(u,v)}",
                              f"Acyclic: {acy_check and in_arc_check}",
                              f"Best_optimal: {self.format_indian_number(round(current_cost))}", 
                              f"New_optimal: {self.format_indian_number(round(self.total_cost))}", 
                              f"Solve_time: {round(self.ampl.get_value('_solve_elapsed_time'), 2)}", 
                              f"Solve_result: {self.solve_result}", "Improved: No ", 
                              f"Time_count: {round(time.time() - self.start_time, 2)}")
                        
                        self.network_graph.remove_edge(v, u)
                        self.network_graph.add_edge(u, v)  
                else:
                    # print("Arc", (u,v),"Acyclic:",acy_check and in_arc_check, "Best optimal: ", '{:,}'.format(round(current_cost)), "New optimal: ", '{:,}'.format(round(self.total_cost)),  "Solve_time:", self.ampl.get_value('_solve_elapsed_time'), "Solve_result: ", self.solve_result)
                    print(f"Arc: {(u,v)}",
                          f"Acyclic: {acy_check and in_arc_check}",
                          f"Best_optimal: {self.format_indian_number(round(current_cost))}", 
                          f"New_optimal: {self.format_indian_number(round(self.total_cost))}", 
                          f"Solve_time: {round(self.ampl.get_value('_solve_elapsed_time'), 2)}", 
                          f"Solve_result: {self.solve_result}", "Improved: No ", 
                          f"Time_count: {round(time.time() - self.start_time, 2)}")
                    self.network_graph.remove_edge(v, u)
                    self.network_graph.add_edge(u, v)                         
            else:
                print(f"Arc: {(u,v)}",
                      f"Acyclic: {acy_check and in_arc_check}")
                
                self.network_graph.remove_edge(v, u)
                self.network_graph.add_edge(u, v)                      
            print(" ")
            if improved:
                break
        return self.best_acyclic_flow, improved, current_cost, self.l, self.q, self.h, best_arc

    
    @staticmethod
    def worker(data, stop_flag):
        # (arc, network_graph) = data
        # verbose=False
        # with contextlib.redirect_stdout(None):
        (arc, network_graph,current_cost, best_acyclic_flow, data_number, l, q, h) = data
        
        u, v = arc
        
        # print(arc)
        # print(ampl_config)
        data_list = [
            "d1_Sample_input_cycle_twoloop",
            "d2_Sample_input_cycle_hanoi",
            "d3_Sample_input_double_hanoi",
            "d4_Sample_input_triple_hanoi",
            "d5_Taichung_input",
            "d6_HG_SP_1_4",
            "d7_HG_SP_2_3",
            "d8_HG_SP_3_4",
            "d9_HG_SP_4_2",
            "d10_HG_SP_5_5",
            "d11_HG_SP_6_3",
            "d12",
            "d13",
            "d14_NewYork",
            "d15_foss_poly_0",
            "d16_foss_iron",
            "d17_foss_poly_1",
            "d18_pescara",
            "d19_modena"
        ]
        # Reverse the arc in a copy of the graph
        network_graph.remove_edge(u, v)
        network_graph.add_edge(v, u)
        
        # Create a new AMPL instance and initialize
        ampl = AMPL()
        ampl.read("../m1Basic.mod")
        ampl.read_data(f"/home/nitishdumoliya/waterNetwork/data/{data_list[data_number]}.dat")
        
        nodes = ampl.getSet('nodes')
        source = ampl.getSet('Source')
        arcs = ampl.getSet('arcs')
        pipes = ampl.getSet('pipes')
        
        # print(nodes)
        # print(arcs)
        # print(ampl_config["current_cost"])
        
        # L = ampl.getParameter('L').to_dict()
        # D = ampl.getParameter('D').to_dict()
        # C = ampl.getParameter('C').to_dict()
        # P = ampl.getParameter('P').to_dict()
        # R = ampl.getParameter('R').to_dict()
        # E = ampl.getParameter('E').to_dict()
        # d = ampl.getParameter('d').to_dict()

        delta = 0.1
        p = 1.852
        omega = 10.67
        
        ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
        # ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}1.2654*l[i,j,k]*(d[k])^1.327;")
        # self.fix_leaf_arc_flow()
        # self.update_initial_points1(ampl_config["l"], ampl_config["q"], ampl_config["h"], ampl_config["t"], ampl_config["all_duals"], ampl_config["inarc"])
        
        for (i, j, k), val in l.items():
            ampl.eval(f'let l[{i},{j},{k}] := {val};')
        for (i, j), val in q.items():
            ampl.eval(f'let q[{i},{j}] := {val};')
        for i, val in h.items():
            ampl.eval(f'let h[{i}] := {val};')
            
        # Add constraints for the arc direction
        if (u, v) in arcs:
            ampl.eval(f"s.t. flow_direction{u}_{v}: q[{u}, {v}]<=0;")
        else:
            ampl.eval(f"s.t. flow_direction{u}_{v}: q[{v}, {u}]>=0;")
            
        ampl.eval("subject to con7{(i,j) in arcs}: -sum{k in nodes diff Source} D[k] <= q[i,j];")
        ampl.eval("subject to con8{(i,j) in arcs}: q[i,j] <= sum{k in nodes diff Source} D[k];")
       
        ampl.option["solver"] = "ipopt"
        ampl.option["snopt_options"] = "major_iterations_limit=1000"
        # ampl.set_option("ipopt_options", "outlev = 0 expect_infeasible_problem = yes bound_push = 0.01 bound_frac = 0.01 warm_start_init_point = yes max_iter = 400")   #max_iter = 1000
        ampl.set_option("ipopt_options", """outlev = 0 expect_infeasible_problem = yes bound_push = 0.01 bound_frac = 0.01 warm_start_init_point = yes mu_strategy = adaptive mu_oracle = loqo max_iter=300 """)   #max_iter = 1000 mu_strategy = adaptive mu_oracle = loqo max_soc = 4
        ampl.option["presolve_eps"] = "6.82e-14"
        ampl.option['presolve'] = 1

        def format_indian_number(num):
            num_str = str(num)
            if len(num_str) <= 3:
                return num_str
            else:
                # Split the number into the last three digits and the rest
                last_three = num_str[-3:]
                remaining = num_str[:-3]
                # Add commas every two digits in the remaining part
                remaining = ','.join([remaining[max(i - 2, 0):i] for i in range(len(remaining), 0, -2)][::-1])
                return remaining + ',' + last_three
                
        
        # Solve the model
        # print(stop_flag.is_set())
        if not stop_flag.is_set():
            
            with contextlib.redirect_stdout(None):
                ampl.solve()
                
            print(f"Arc: {(u,v)}",
              f"Acyclic: True",
              f"Best_optimal: {format_indian_number(round(current_cost))}", 
              f"New_optimal: {format_indian_number(round(ampl.get_objective("total_cost").value()))}", 
              f"Solve_time: {round(ampl.get_value('_solve_elapsed_time'), 2)}", 
              f"Solve_result: {ampl.get_value("solve_result")}",
              f"Improved: {ampl.get_objective("total_cost").value() < current_cost}")

            # indegree_2_or_more = [node for node, indeg in network_graph.in_degree() if indeg >= 2]
            # super_source_out_arc = self.fix_arc_set()
            # all_duals = {}
            # for con_name, val in ampl.get_constraints():
            #     # Get dual values for each constraint
            #     dual_values = val.getValues()
            #     all_duals[con_name] = dual_values
    
            # inarc = []
            # for node in indegree_2_or_more:
            #     for (u, v) in network_graph.in_edges(node):
            #         if (u, v) in arcs:
            #             if (u,v) not in self.super_source_out_arc:
            #                 inarc.append((u,v))
            #         else:
            #             inarc.append((v,u))
                    

            # # Check if the stop flag is already set
            # if stop_flag.is_set():
            #     print(f"Worker for arc {arc} stopped as condition was met.")
            #     return {
            #         "total_cost": ampl.get_objective("total_cost").value(),
            #         "solve_result": ampl.get_value("solve_result"),
            #         "solve_time": round(ampl.get_value('_solve_elapsed_time'), 2),
            #         "improved": ampl.get_objective("total_cost").value() < current_cost,
            #         "acyclic_flow": network_graph.copy(), 
            #         "l": l,
            #         "q": q,
            #         "h": h,
            #     }
            
            # Check solution status
            if ampl.get_value("solve_result") == "solved" or ampl.get_value("solve_result") == "solved?":
                l = ampl.getVariable("l").getValues().to_dict()
                q = ampl.getVariable("q").getValues().to_dict()
                h = ampl.getVariable("h").getValues().to_dict()
                
                if ampl.get_objective("total_cost").value()<current_cost:
                    print(f"Condition met for arc {arc}, stopping parallel run.")
                    stop_flag.set()  # Set the stop flag
    
                return {
                    "arc":(v,u),
                    "total_cost": ampl.get_objective("total_cost").value(),
                    "solve_result": ampl.get_value("solve_result"),
                    "solve_time": round(ampl.get_value('_solve_elapsed_time'), 3),
                    "improved": ampl.get_objective("total_cost").value() < current_cost,
                    "acyclic_flow": network_graph.copy(), 
                    "l": l,
                    "q": q,
                    "h": h,
                    "nlp_run":True
                }
            else:
                return {
                    "arc":(v,u),
                    "total_cost": ampl.get_objective("total_cost").value(),
                    "solve_result": ampl.get_value("solve_result"),
                    "solve_time": round(ampl.get_value('_solve_elapsed_time'), 3),
                    "improved": ampl.get_objective("total_cost").value() < current_cost,
                    "acyclic_flow": network_graph.copy(), 
                    "l": l,
                    "q": q,
                    "h": h,
                    "nlp_run":True
                }
        return {
                "arc":(v,u),
                "total_cost": 'inf',
                "solve_result": "not_solved",
                "solve_time": 0,
                "improved": False,
                "acyclic_flow": None, 
                "l": None,
                "q": None,
                "h": None,
                "nlp_run":False
            }
                
    def run_parallel(self, network_graph, ampl_config):
        best_result = None
        improved = False
        
        
        current_cost =  ampl_config["current_cost"]
        best_acyclic_flow =  ampl_config["best_acyclic_flow"]
        l =  ampl_config["l"]
        q =  ampl_config["q"]
        h =  ampl_config["h"]
        
        # t =  ampl_config["t"]
        # all_duals =  ampl_config["all_duals"]
        # inarc =  ampl_config["inarc"]
        
        # Prepare the data for each worker
        # acyclic_arc_set = list(network_graph.edges)
        # data = [(arc, network_graph.copy(), ampl_config) for arc in self.acyclic_arc_set]
        
        # global stop_flag, pool
        
        # Initialize the shared stop flag
        # stop_flag = multiprocessing.Value('i', 0)  # Shared flag to stop workers
        
        # def check_result(result):
        #     """
        #     Callback function to check the worker's result.
        #     If the condition is met, set the stop flag to True.
        #     """
        #     global stop_flag
        #     if result["total_cost"] < current_cost:
        #         stop_flag.value = True  # Stop further processing
        #         print(f"Condition met. Stopping parallel computation. Total cost: {result['total_cost']}")
        #         pool.terminate()  # Terminate the pool immediately
                
        data = [(arc, network_graph.copy(),current_cost, best_acyclic_flow, self.data_number, l, q, h ) for arc in self.sorted_delta_arc]
        
        # print(data)
        # Run workers in parallel
        # with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        #     results = pool.map(self.worker, data)
        
        # nproc = os.cpu_count()
        # print(nproc, "processors available")
        # print()
        pool = multiprocessing.Pool(8)
        # results = pool.map(self.worker, data)
        # Close the pool and wait for all tasks to complete
        # pool.close()
        # pool.join()
        
        # try:
        #     results = pool.starmap(self.worker, [(d, stop_flag) for d in data] )
        # finally:
        #     pool.close()
        #     pool.join()
        
        # Use a Manager to create a shared stop_flag
        # with multiprocessing.Manager() as manager:
        #     stop_flag = manager.Event()  # Shared event object
    
        #     # Create the pool of workers
        #     with multiprocessing.Pool(processes=8) as pool:
        #         try:
        #             results = pool.starmap(self.worker, [(d, stop_flag) for d in data])
        #         finally:
        #             pool.close()
        #             pool.join()
    
        # with multiprocessing.Manager() as manager:
        #     stop_flag = manager.Event()  # Shared event object
    
        #     with multiprocessing.Pool(processes=4) as pool:
        #         results = []
    
        #         # Use `imap_unordered()` instead of `starmap()` starmap_async
        #         for result in pool.starmap_async(self.worker, [(d, stop_flag) for d in data]):
        #             if stop_flag.is_set():  
        #                 pool.terminate()  # Kill all remaining processes
        #                 break  # Exit the loop
                
        #         pool.close()
        #         pool.join()
    
        #         print("Parallel execution stopped.")

        with multiprocessing.Manager() as manager:
            stop_flag = manager.Event()  # Shared event object

            with multiprocessing.Pool(processes=8) as pool:
                try:
                    # results = pool.starmap_async(self.worker, [(d, stop_flag) for d in data])  #apply_async, starmap_async
                    results = [pool.apply_async(self.worker, args=(d, stop_flag)) for d in data]

                    # Monitor if stop_flag is set
                    # while not results.ready():
                    #     if stop_flag.is_set():
                    #         pool.terminate()  # Stop all processes
                    #         break
                    #     time.sleep(0.5)  # Avoid busy-waiting
                    # results = results.get()

                    while not all(res.ready() for res in results):
                        if stop_flag.is_set():
                            pool.terminate()
                            break
                        time.sleep(0.5)
                    
                    results = [res.get() for res in results if res.ready()]  # Retrieve only ready results
                    
                    pool.close()
                    pool.join()

                    print("Parallel execution stopped.")
                
                except Exception as e:
                    print(f"Error: {e}")
                    pool.terminate()
                    pool.join()
                    
        # Filter valid results
        valid_results = [r for r in results if r["solve_result"] == "solved" or r["solve_result"] == "solved?"]
        # nlp_list = [r for r in results if r["nlp_run"] == True]
        # self.number_of_nlp += len(nlp_list)
        
        # solve_time = 0
        # for i in range(len(nlp_list)):
        #     solve_time += nlp_list[i]["solve_time"] 
            
        # self.solver_time +=  solve_time
            
        if valid_results:
            # Find the best result
            best_result = min(valid_results, key=lambda r: r["total_cost"])
            
            # print(best_result)
            
            if best_result["total_cost"] < ampl_config["current_cost"]:
                ampl_config["current_cost"] = best_result["total_cost"]
                ampl_config["best_acyclic_flow"] = best_result["acyclic_flow"]
                ampl_config["l"] = best_result["l"]
                ampl_config["q"] = best_result["q"]
                ampl_config["h"] = best_result["h"]
                # ampl_config["t"] = best_result["t"]
                # ampl_config["all_duals"]: best_result["all_duals"]
                # ampl_config["inarc"]: best_result["inarc"]
                improved = True
                ampl_config["best_acyclic_flow"] = best_result["acyclic_flow"].copy()
                network_graph = best_result["acyclic_flow"].copy()
                arc = best_result["arc"]
                print("arc", arc)
                print(f"New best solution found! Total cost: {ampl_config['current_cost']}")
            else:
                improved = False
                arc = best_result["arc"]
                # print("arc", arc)
                print("No improved solutions found in this iteration.")
        # else:
        #     print("No improved solutions found in this iteration.")

        # # Stop if no improvement
        # if not improved:
        #     break

        # Update the graph for the next iteration
        
        return network_graph, ampl_config, improved, arc

    def iterate_acyclic_flows(self):
        """Iterate to find improved acyclic flows by attempting arc reversals while maintaining acyclicity."""
        improved = True 
        
        self.best_acyclic_flow = self.network_graph.copy()
        
        # self.indegree_2_or_more = [node for node, indeg in self.best_acyclic_flow.in_degree() if indeg >= 2]
        # print("indegree_2_or_more:", self.indegree_2_or_more)
        
        if self.solve_result == "solved":
            current_cost = self.total_cost
            self.l = self.ampl.getVariable('l').getValues().to_dict()
            self.q = self.ampl.getVariable('q').getValues().to_dict()
            self.h = self.ampl.getVariable('h').getValues().to_dict()
            self.t = self.ampl.getVariable('t').getValues().to_dict()
            
            self.all_duals = {}
            for con_name, val in self.ampl.get_constraints():
                # Get dual values for each constraint
                dual_values = val.getValues()
                self.all_duals[con_name] = dual_values
        
        elif self.solve_result != "solved":
            current_cost = 10e+14
            self.l = None
            self.q = None
            self.h = None
        
        iteration = 1
        best_arc = None
        
        self.inarc = []
        for node in self.indegree_2_or_more:
            for (u, v) in self.best_acyclic_flow.in_edges(node):
                if (u, v) in self.arcs:
                    if (u,v) not in self.super_source_out_arc:
                        self.inarc.append((u,v))
                else:
                    self.inarc.append((v,u))
        
        # self.plot_graph(self.super_source_out_arc, current_cost, 0, self.q, self.h, self.D, (0,0), self.l, self.C)
        self.network_graph = self.best_acyclic_flow.copy()
        self.acyclic_arc_set = self.acyclic_arcs()
        sum_arc = {}       
        delta_arc = {}
        for node in self.indegree_2_or_more:
            sum_arc[node]=0
            for (u,v) in self.best_acyclic_flow.in_edges(node):
                if (u,v) in self.arcs:
                    sum_arc[node] += abs(self.q[u,v])
                else:
                    sum_arc[node] += abs(self.q[v,u])
            
            for (u,v) in self.best_acyclic_flow.in_edges(node):
                # delta_arc[u,v] = 0
                if (u,v) in self.arcs:
                    if (u,v) in self.acyclic_arc_set:
                        # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[u,v]))
                        delta_arc[u,v] = self.D[node] - sum_arc[node] + abs(self.q[u,v])
                else:
                    if (u,v) in self.acyclic_arc_set:
                        # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[v,u]))
                        delta_arc[u, v] = self.D[node] - sum_arc[node] + abs(self.q[v,u])
                    
        sorted_dict = dict(sorted(delta_arc.items(), key=lambda item: abs(item[1]), reverse=True))
        
        # print(sorted_dict)
        self.sorted_delta_arc = list(sorted_dict.keys())
        print("sorted_delta_arc:",self.sorted_delta_arc)
        # ampl_config = {
        #     "current_cost": current_cost,
        #     "best_acyclic_flow": self.network_graph,
        #      "l": self.l,
        #      "q": self.q,
        #      "h": self.h,
        #      "t": self.t,
        #      "all_duals": self.all_duals,
        #     "inarc": self.inarc
        # }
        
        ampl_config = {
            "current_cost": current_cost,
            "best_acyclic_flow": self.network_graph,
            "l": self.l,
            "q": self.q,
            "h": self.h,
        }
        
        while improved and self.sorted_delta_arc:
            print("\n******************************************************************************************************************************************")
            print("Iteration :",iteration, "\n")
            print(self.sorted_delta_arc)
            # self.best_acyclic_flow, improved, current_cost, self.l, self.q, self.h, best_arc = self.iterate_arc(iteration, improved, current_cost, best_arc)
            self.network_graph, ampl_config, improved, arc = self.run_parallel(self.network_graph, ampl_config)
            self.best_acyclic_flow = self.network_graph
            self.acyclic_arc_set = self.acyclic_arcs()
            # print("acyclic_arc_set:",acyclic_arc_set)
            sum_arc = {}       
            delta_arc = {}
            self.indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
            print("self.indegree_2_or_more", self.indegree_2_or_more)
            for node in self.indegree_2_or_more:
                sum_arc[node]=0
                for (u,v) in self.best_acyclic_flow.in_edges(node):
                    if (u,v) in self.arcs:
                        sum_arc[node] += abs(self.q[u,v])
                    else:
                        sum_arc[node] += abs(self.q[v,u])
                
                for (u,v) in self.best_acyclic_flow.in_edges(node):
                    # delta_arc[u,v] = 0
                    if (u,v) in self.arcs:
                        if (u,v) in self.acyclic_arc_set:
                            # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[u,v]))
                            delta_arc[u,v] = self.D[node] - sum_arc[node] + abs(self.q[u,v])
                    else:
                        if (u,v) in self.acyclic_arc_set:
                            # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[v,u]))
                            delta_arc[u, v] = self.D[node] - sum_arc[node] + abs(self.q[v,u])
                        
            sorted_dict = dict(sorted(delta_arc.items(), key=lambda item: abs(item[1]), reverse=True))
            
            # print(sorted_dict)
            self.sorted_delta_arc = list(sorted_dict.keys())
            
            if improved:
                (i,j) = arc
                self.sorted_delta_arc.remove((i,j))
            print("sorted_delta_arc:",self.sorted_delta_arc)
            # print(self.acyclic_arc_set)
            
            # print("Current best acyclic network:")
            # plt.figure(figsize=(10, 8))
            # nx.draw_spectral(best_acyclic_flow, with_labels=True)
            # plt.show()
            # super_source_out_arc = self.fix_arc()
            # super_source_out_arc.append(best_arc)
            iteration += 1
            # print(f"Current best solution: {current_cost}")
            # print(" ")
        
        print("\n*********************************************************Final best results***************************************************************")
        print("Water Network:", self.data_number)
        print(f"Final best objective: {ampl_config["current_cost"]}")

        # print("Number of nlp problem solved:", self.number_of_nlp)
        print("Total number of iteration to solve the problem:", iteration-1)
        # print("length of the arcs: ", l, "\n")
        # print("flow in the arcs: ", q, "\n")
        # print("head value at node: ", h, "\n")
        # self.network_graph = best_acyclic_flow
        # self.load_model()
        # self.update_model()
        # self.multi_solve(current_cost)
        # self.ampl.close()
    
    # Function to suppress output
    @contextlib.contextmanager
    def suppress_output(self):
        # Open devnull to suppress the output
        with open(os.devnull, 'w') as devnull:
            # Redirect stdout and stderr
            old_stdout = sys.stdout
            old_stderr = sys.stderr
            sys.stdout = devnull
            sys.stderr = devnull
            try:
                yield
            finally:
                # Restore original stdout and stderr
                sys.stdout = old_stdout
                sys.stderr = old_stderr
    
    def solve(self):
        with self.suppress_output():
            """Solve the optimization problem."""
            self.ampl.option["solver"] = "ipopt"
            self.ampl.set_option("ipopt_options", "outlev = 0 expect_infeasible_problem = yes bound_push = 0.001 bound_frac = 0.001 warm_start_init_point = yes")   #max_iter = 1000
            # self.ampl.set_option("ipopt_options", """outlev = 0 expect_infeasible_problem = yes bound_push = 0.01 bound_frac = 0.01 warm_start_init_point = yes mu_strategy = adaptive mu_oracle = loqo  """)   #max_iter = 1000 mu_strategy = adaptive mu_oracle = loqo max_soc = 4 halt_on_ampl_error = yes
            self.ampl.option["presolve_eps"] = "6.82e-14"
            self.ampl.option['presolve'] = 1
            # self.ampl.option['solver_msg'] = 0
            # self.ampl.option['show_stats'] = 0
            self.ampl.solve()
            self.solve_result = self.ampl.solve_result
            self.total_cost = self.ampl.get_objective("total_cost").value()
        # print("Objective:", self.total_cost)
        # print("solve_result: ",self.solve_result)
        solve_time = self.ampl.get_value('_solve_elapsed_time')
        self.solver_time += solve_time
        self.number_of_nlp += 1
        
    def run(self):
        """Main method to run the optimization process."""
        
        self.start_time = time.time()
        print("Solve the original nonconvex optimization problem using IPOPT ")
        self.load_model()
        self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
        # self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}1.2654*l[i,j,k]*(d[k])^1.327;")
        self.fix_leaf_arc_flow()
        
        self.super_source_out_arc = self.fix_arc_set()
        
        self.ampl.eval("subject to con7{(i,j) in arcs}: -sum{k in nodes diff Source} D[k] <= q[i,j];")
        self.ampl.eval("subject to con8{(i,j) in arcs}: q[i,j] <= sum{k in nodes diff Source} D[k];")
       
        # self.ampl.eval("display option ipopt_options;")     # Display all options in AMPL

        self.solve()
        
        print("Objective: ",self.total_cost)
        print("Solve_result: ",self.solve_result)
        print("Solve_time:", self.ampl.get_value('_solve_elapsed_time'),"\n")
        
        self.l = self.ampl.getVariable('l').getValues().to_dict()
        self.q = self.ampl.getVariable('q').getValues().to_dict()
        self.h = self.ampl.getVariable('h').getValues().to_dict()
        
        self.super_source_out_arc = self.fix_arc_set()
        self.network_graph = self.generate_random_acyclic_from_solution(self.q)
        
        # print("Fix the flow direction in optimization model and solve the updated model")
        
        self.inarc = []
        self.indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
        
        for node in self.indegree_2_or_more:
            for (u, v) in list(self.network_graph.in_edges(node)):
                if (u, v) in self.arcs:
                    self.inarc.append((u,v))
                else:
                    self.inarc.append((v,u))
        # print("inarc:", self.inarc)
        inarc_ = self.inarc
        
        inarc_set = []
        for (i, j) in self.inarc:
            if (i, j) in self.arcs:
                inarc_set.append(f"({i},{j})")
            else:
                inarc_set.append(f"({j},{i})")
        inarc_set = ", ".join(inarc_set)
        
        # self.display_results()
        self.iterate_acyclic_flows()
        elapsed_time = time.time() - self.start_time
        
        print("Solver_time:",self.solver_time, "seconds")
        print(f"Heuristic elapsed time:, {elapsed_time} seconds = {elapsed_time/60:.2f} minutes.")
        
if __name__ == "__main__":
    data_list = [
        "d1_Sample_input_cycle_twoloop",
        "d2_Sample_input_cycle_hanoi",
        "d3_Sample_input_double_hanoi",
        "d4_Sample_input_triple_hanoi",
        "d5_Taichung_input",
        "d6_HG_SP_1_4",
        "d7_HG_SP_2_3",
        "d8_HG_SP_3_4",
        "d9_HG_SP_4_2",
        "d10_HG_SP_5_5",
        "d11_HG_SP_6_3",
        "d12",
        "d13",
        "d14_NewYork",
        "d15_foss_poly_0",
        "d16_foss_iron",
        "d17_foss_poly_1",
        "d18_pescara",
        "d19_modena"
    ]

    # Select the data number here (0 to 18)
    data_number = 8
    input_data_file = f"/home/nitishdumoliya/waterNetwork/data/{data_list[data_number]}.dat"
    print("Water Network:", data_list[data_number],"\n")
    optimizer = WaterNetworkOptimizer("../m1Basic.mod", input_data_file, data_number, verbose=True)
    # optimizer = WaterNetworkOptimizer(sys.argv[1], sys.argv[3], sys.argv[2])
    optimizer.run()

Water Network: d9_HG_SP_4_2 

Solve the original nonconvex optimization problem using IPOPT 
Objective:  9056439.290930092
Solve_result:  solved
Solve_time: 13.939031 

sorted_delta_arc: [(33, 49), (25, 49), (126, 59), (37, 5), (114, 5), (32, 60), (67, 74), (43, 85), (29, 112), (69, 42), (22, 9), (80, 6), (29, 9), (44, 35), (24, 101), (36, 42), (105, 112), (55, 100), (77, 87), (113, 135), (141, 134), (3, 138), (91, 87), (51, 103), (117, 143), (108, 105), (75, 36), (8, 24), (53, 63), (95, 51), (111, 28), (26, 74), (31, 104), (115, 138), (11, 126), (60, 57), (60, 58), (5, 76), (76, 126), (119, 115), (59, 135), (93, 54), (6, 85), (73, 28), (87, 63), (103, 105), (89, 2), (74, 60), (99, 24), (91, 47), (70, 101), (10, 43), (47, 53), (28, 51), (23, 65), (53, 78), (48, 2), (41, 17), (11, 48), (101, 104), (82, 109), (39, 83), (96, 47), (40, 43), (21, 68), (77, 57), (92, 114), (109, 89), (18, 23), (4, 50), (52, 109), (61, 93), (56, 54), (123, 84), (19, 84)]

*************************************

KeyboardInterrupt: 

In [3]:
import networkx as nx
from amplpy import AMPL
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
import sys
import os
import contextlib
from sklearn.decomposition import PCA
import random

import matplotlib.lines as mlines
import matplotlib.patches as mpatches

import warnings
warnings.filterwarnings("ignore")

import multiprocessing
# !pip install pathos

# from pathos.multiprocessing import ProcessingPool as Pool

class WaterNetworkOptimizer:
    def __init__(self, model_file, data_file, data_number, verbose=True):
        self.ampl = AMPL()
        self.model_file = model_file
        self.data_file = data_file
        self.data_number = data_number
        self.total_cost = None
        self.network_graph = None
        self.solve_result = None
        self.solver_time = 0
        self.best_acyclic_flow = None
        self.number_of_nlp = 0
        self.verbose = verbose

    def load_model(self):
        """Load the model and data."""
        self.ampl.reset()
        self.ampl.read(self.model_file)
        self.ampl.read_data(self.data_file)
        
        self.nodes = self.ampl.getSet('nodes')
        self.source = self.ampl.getSet('Source')
        self.arcs = self.ampl.getSet('arcs')
        self.pipes = self.ampl.getSet('pipes')
        
        self.L = self.ampl.getParameter('L').to_dict()
        self.D = self.ampl.getParameter('D').to_dict()
        self.C = self.ampl.getParameter('C').to_dict()
        self.P = self.ampl.getParameter('P').to_dict()
        self.R = self.ampl.getParameter('R').to_dict()
        self.E = self.ampl.getParameter('E').to_dict()
        self.d = self.ampl.getParameter('d').to_dict()

        self.delta = 0.1
        self.p = 1.852
        self.omega = 10.67

    def create_digraph(self):
        nodes_list = [i for i in self.ampl.getSet('nodes')]
        edges_list = self.ampl.getSet('arcs').to_list()
        self.network_graph = nx.DiGraph()
        self.network_graph.add_nodes_from(nodes_list)
        self.network_graph.add_edges_from(edges_list)
        print(nodes_list)
        print(edges_list)

    def display_results(self):
        """Display relevant results from the optimization."""
        self.ampl.eval("display {(i,j) in arcs, k in pipes:l[i,j,k]>1} l[i,j,k];")
        self.ampl.eval("display {(i,j) in arcs}: q[i,j];")
        self.ampl.eval("display h;")
        self.ampl.eval("display solve_result;")
        self.total_cost = self.ampl.get_objective("total_cost").value()
        print("Objective:", self.total_cost)
        # self.ampl.eval("display {(i,j) in arcs} h[i] - h[j];")
        # self.ampl.eval("display {i in nodes} h[i] - (E[i] + P[i]);")

    # def plot_graph(self):
    #     print("Edges of the graph:",self.network_graph.edges())
    #     plt.figure(figsize=(10, 8))
    #     nx.draw_spectral(self.network_graph, with_labels=True)
    #     plt.show()


    def plot_graph1(self, super_source_out_arc=None, best_arc=None,current_cost = None, iteration= 1):
        # print("Edges of the graph:", self.network_graph.edges())
        indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
        
        plt.figure(figsize=(10, 8))
        pos = nx.spectral_layout(self.network_graph)
        nx.draw_networkx_nodes(self.network_graph, pos, node_color='lightblue', node_size=200)
        
        if indegree_2_or_more:
            nx.draw_networkx_nodes(self.network_graph, pos, nodelist=indegree_2_or_more, node_color='orange', node_size=200)
            
        nx.draw_networkx_labels(self.network_graph, pos)
        nx.draw_networkx_edges(self.network_graph, pos, edge_color='black')
        
        if super_source_out_arc:
            nx.draw_networkx_edges(self.network_graph, pos, edgelist=super_source_out_arc, edge_color='red', width=1)
            
        if best_arc:
            nx.draw_networkx_edges(self.network_graph, pos, edgelist=[best_arc], edge_color='magenta', width=1)
            
        plt.title(f"total_cost:{self.total_cost}")
        plt.savefig(f"/home/nitishdumoliya/waterNetwork/model/figure/d{self.data_number}_iteration_{iteration}.png")
        plt.box(False)
        plt.show()

    def plot_graph(self, super_source_out_arc=None, current_cost = None, iteration = 1, edge_weights= None, h = None, D = None, arc=(0,0), l ={}, C = {}):
        # self.network_graph = nx.DiGraph()
        # self.network_graph.add_edges_from(edges)
        # Node positions as per your specifications
        
        new_positions0 = {
             2 : (2000.00, 3000.00),
             3 : (1000.00, 3000.00),        
             4 : (2000.00, 2000.00),        
             5 : (1000.00, 2000.00),       
             6 : (2000.00, 1000.00),       
             7 : (1000.00, 1000.00),      
             1 : (3000.00, 3000.00)     
            }  
        
        new_positions1 = {
            2: (6000.00, 2000.00),
            3: (6000.00, 4000.00),
            4: (7500.00, 4000.00),
            5: (9000.00, 4000.00),
            6: (10000.00, 4000.00),
            7: (10000.00, 6000.00),
            8: (10000.00, 8000.00),
            9: (10000.00, 10000.00),
            10: (9000.00, 10000.00),
            11: (9000.00, 11500.00),
            12: (9000.00, 13000.00),
            13: (7000.00, 13000.00),
            14: (8000.00, 10000.00),
            15: (7000.00, 10000.00),
            16: (6000.00, 10000.00),
            17: (6000.00, 8500.00),
            18: (6000.00, 7000.00),
            19: (6000.00, 5500.00),
            20: (4500.00, 4000.00),
            21: (4500.00, 2000.00),
            22: (4500.00, 0.00),
            23: (3000.00, 4000.00),
            24: (3000.00, 7000.00),
            25: (3000.00, 10000.00),
            26: (4000.00, 10000.00),
            27: (5000.00, 10000.00),
            28: (1500.00, 4000.00),
            29: (0.00, 4000.00),
            30: (0.00, 7000.00),
            31: (0.00, 10000.00),
            32: (1500.00, 10000.00),
            1: (8000.00, 0.00)
        }       
        
        new_positions16 = {
            2: (5679.61, 9538.83),
            3: (4862.46, 9538.83),
            4: (2750.81, 9474.11),
            5: (1852.75, 8357.61),
            6: (1974.11, 6076.05),
            7: (1974.11, 5149.68),
            8: (4235.44, 5076.86),
            9: (6411.81, 5093.04),
            10: (5412.62, 7888.35),
            11: (4510.52, 8264.56),
            12: (3033.98, 9243.53),
            13: (2301.78, 8078.48),
            14: (2944.98, 7669.90),
            15: (3786.41, 7139.97),
            16: (4830.10, 6480.58),
            17: (7099.51, 8438.51),
            18: (5505.66, 8450.65),
            19: (3563.92, 8839.00),
            20: (3167.48, 7532.36),
            21: (2730.58, 7285.60),
            22: (3511.33, 6666.67),
            23: (4097.90, 6286.41),
            24: (3337.38, 5121.36),
            25: (4530.74, 6011.33),
            26: (4215.21, 7783.17),
            27: (5194.17, 7055.02),
            28: (5218.45, 5089.00),
            29: (5622.98, 5999.19),
            30: (5950.65, 5796.93),
            31: (6614.08, 7621.36),
            32: (5380.26, 7544.50),
            33: (6318.77, 7281.55),
            34: (6549.35, 7212.78),
            35: (6585.76, 6092.23),
            36: (7152.10, 6104.37),
            1: (7111.65, 7532.36),
            37: (7669.90, 7783.17)
        }

        # # Update node positions
        pos = new_positions0
        # pos = nx.spectral_layout(self.network_graph)

        cost = {}

        for (i,j) in self.ampl.getSet('arcs'):
            cost[i,j] = sum(l[i,j,k] * C[k] for k in self.ampl.getSet('pipes'))
        
        plt.figure(figsize=(10, 8))
        cmap = plt.cm.plasma

        nx.draw_networkx_nodes(self.network_graph, pos, node_color='lightblue', node_size=300, label='Regular Nodes')

        indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
        if indegree_2_or_more:
            nx.draw_networkx_nodes(self.network_graph, pos, nodelist=indegree_2_or_more, node_color='orange', node_size=300, label='Nodes with In-Degree ≥ 2')

        nx.draw_networkx_labels(self.network_graph, pos, font_size=10)

        nx.draw_networkx_edges(self.network_graph, pos, arrowstyle="->", arrowsize=12, edge_color='gray', label='Regular Arcs')

        if super_source_out_arc:
            nx.draw_networkx_edges(self.network_graph, pos, edgelist=super_source_out_arc,arrowstyle="->", arrowsize=12, edge_color='red', width=1, label='Fix arc direction')

        # if best_arc:
        #     nx.draw_networkx_edges(self.network_graph, pos, edgelist=[best_arc],arrowstyle="->", arrowsize=12, edge_color='magenta', width=1, label = 'Best arc')
        # Annotate node demands
        if h:
            for node, (x, y) in pos.items():
                demand = h.get(node, 0)  # Get the head for the node, default to 0 if not in dictionary
                plt.text(x, y + 250, f"{demand:.2f}", fontsize=8, color='blue', ha='center')  # Annotate demand below the node
                # plt.text(x, y + 100, f"{demand:.2f}", fontsize=8, color='blue', ha='center')  # Annotate demand below the node
                # plt.text(x, y -100 , f"{demand:.2f}", fontsize=8, color='magenta', ha='center')  # Annotate demand below the node
                # plt.text(mid_x, mid_y + 50 , f"{weight:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
                # plt.text(mid_x, mid_y + 100, f"{value:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        if D:
            for node, (x, y) in pos.items():
                demand = D.get(node, 0)  # Get the demand for the node, default to 0 if not in dictionary
                plt.text(x, y - 300, f"{demand:.2f}", fontsize=8, color='magenta', ha='center')  # Annotate demand below the node
                # plt.text(x, y -100 , f"{demand:.2f}", fontsize=8, color='magenta', ha='center')  # Annotate demand below the node
            

        if edge_weights:
            for (u, v), weight in edge_weights.items():
                # if self.network_graph.has_edge(u, v):
                mid_x = (pos[u][0] + pos[v][0]) / 2  # Midpoint x-coordinate
                mid_y = (pos[u][1] + pos[v][1]) / 2  # Midpoint y-coordinate
                # plt.text(mid_x, mid_y+100 , f"{weight:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
                plt.text(mid_x, mid_y + 50 , f"{weight:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        if cost:
            for (u, v), value in cost.items():
                # if self.network_graph.has_edge(u, v):
                mid_x = (pos[u][0] + pos[v][0]) / 2  # Midpoint x-coordinate
                mid_y = (pos[u][1] + pos[v][1]) / 2  # Midpoint y-coordinate
                # plt.text(mid_x, mid_y-300 , f"{round(value)}", fontsize=8, color='purple', ha='center')  # Annotate weight on edge
                # plt.text(mid_x, mid_y + 100, f"{value:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        pipe_dia_arc = {}
        for (i,j) in self.arcs:
            list_=[]
            for k in self.pipes:
                if l[i,j,k]>= 1e-5:
                    list_.append(k)
            pipe_dia_arc[i,j] = list_
        
        if pipe_dia_arc:
            for (u, v), weight in pipe_dia_arc.items():
                # if self.network_graph.has_edge(u, v):
                mid_x = (pos[u][0] + pos[v][0]) / 2  # Midpoint x-coordinate
                mid_y = (pos[u][1] + pos[v][1]) / 2  # Midpoint y-coordinate
                # plt.text(mid_x, mid_y+550 , f"{weight}", fontsize=8, color='purple', ha='center')  # Annotate weight on edge
                plt.text(mid_x, mid_y+150 , f"{weight}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        
        regular_node_patch = mpatches.Patch(color='lightblue', label='Regular Nodes')
        indegree_node_patch = mpatches.Patch(color='orange', label='Nodes with In-Degree ≥ 2')
        regular_edge_line = mlines.Line2D([], [], color='black', label='Regular Arcs')
        super_source_edge_line = mlines.Line2D([], [], color='red', label='Fix arc direction')
        # best_edge_line = mlines.Line2D([], [], color='magenta', label='Best Arc')
        plt.legend(handles=[regular_node_patch, indegree_node_patch, regular_edge_line, super_source_edge_line], loc='lower right')
        
        cost = round(self.total_cost)
        # res = f"{cost:,}"
        plt.title(f"total_cost:{self.format_indian_number(cost)}")
        # (u,v) = arc
        # plt.savefig(f"/home/nitishdumoliya/waterNetwork/model/figure/d{self.data_number}_iteration_{iteration}.png")
        plt.box(True)
        plt.show()

    
    def cycle_basis(self):
        root = self.ampl.getSet('Source').to_list()[0]
        nodes_list = [i for i in self.ampl.getSet('nodes')]
        edges_list = self.ampl.getSet('arcs').to_list() 
        uwg = nx.Graph()
        uwg.add_nodes_from(nodes_list)
        uwg.add_edges_from(edges_list)
        # print("Edges in the undirected graph:", edges_list)
        print("cycle basis for given water network: ",nx.cycle_basis(uwg, root))
        
    def generate_random_acyclic_from_solution(self, q):
        # print("Generate the acyclic network using ipopt solution")
        
        self.network_graph = nx.DiGraph()
        self.network_graph.add_nodes_from(self.nodes)
        
        # q = self.ampl.getVariable('q').getValues().toDict()
        for (i,j) in self.arcs:
            if q[i,j] >= 0:
                self.network_graph.add_edge(i,j)
            else:
                self.network_graph.add_edge(j,i)
        
        return self.network_graph

    
    def generate_random_acyclic_graph(self):
        uwg = nx.Graph()
        nodes_list = [i for i in self.ampl.getSet('nodes')]
        edges_list = self.ampl.getSet('arcs').to_list()
        uwg.add_nodes_from(nodes_list)
        uwg.add_edges_from(edges_list)
        print("Edges in the undirected graph:", edges_list)
        
        # Generate a random spanning tree using Wilson's algorithm
        random_tree = nx.random_spanning_tree(uwg)
        
        # Retrieve the root from the AMPL source set
        root_l = self.ampl.getSet('Source').to_list()
        root = root_l[0]
        print("Root node:", root)

        # Ensure the root is present in the random tree
        if root not in random_tree.nodes:
            raise ValueError("The specified root must be a node in the graph.")

        # Create a directed graph from the random tree starting from the specified root
        self.network_graph = nx.DiGraph()
        visited = set()

        def dfs(node):
            visited.add(node)
            for neighbor in random_tree.neighbors(node):
                if neighbor not in visited:
                    self.network_graph.add_edge(node, neighbor) 
                    dfs(neighbor)

        # Start DFS from the specified root
        dfs(root)

        # Draw the initial directed tree
        plt.figure(figsize=(15, 10))
        plt.subplot(121)
        nx.draw_spectral(self.network_graph, with_labels=True, node_color='lightgreen', font_weight='bold', arrows=True)
        plt.title("Directed Spanning Tree")

        # Add remaining edges from the original graph and check for cycles
        for u, v in uwg.edges():
            if not self.network_graph.has_edge(u, v):  
                self.network_graph.add_edge(u, v)  
                if not nx.is_directed_acyclic_graph(self.network_graph):  
                    self.network_graph.remove_edge(u, v)  
                    self.network_graph.add_edge(v, u)  

        # Draw the final directed graph after adding remaining edges
        plt.subplot(122)
        nx.draw_spectral(self.network_graph, with_labels=True, node_color='lightgreen', font_weight='bold', arrows=True)
        plt.title("Acyclic Directed Graph")
        plt.show()

    def update_model(self):
        # print("Fix the arcs direction using the acyclic network\n")
        edges_list = [(arc[0],arc[1]) for arc in self.ampl.getSet('arcs')]
        for edge in self.network_graph.edges:
            i, j = edge
            if edge in edges_list:
                self.ampl.eval(f"s.t. flow_direction{i}_{j}: q[{i},{j}] >=0;")
                # self.ampl.eval(f"s.t. head_bound_left{i}_{j}: E[{j}]+P[{j}] <= h[{j}];")
                # self.ampl.eval(f"s.t. head_bound_right{i}_{j}: E[{j}] + P[{j}] <= h[{i}];")
            else:
                self.ampl.eval(f"s.t. flow_direction{i}_{j}: q[{j},{i}] <=0;")
                # self.ampl.eval(f"s.t. head_bound_left{i}_{j}: E[{i}]+P[{i}] <= h[{i}];")
                # self.ampl.eval(f"s.t. head_bound_right{i}_{j}: E[{i}] + P[{i}] <= h[{j}];")  


    def is_valid_edge(self, source, target):
        """Check if adding the directed edge (source -> target) maintains acyclicity."""
        self.network_graph.add_edge(source, target)  # Temporarily add the edge
        is_dag = nx.is_directed_acyclic_graph(self.network_graph)  # Check for acyclicity
        self.network_graph.remove_edge(source, target)  # Remove the edge after checking
        return is_dag  # Return True if it maintains acyclicity     
    
    def check_incoming_arcs(self):
        root = list(self.ampl.getSet('Source'))[0]
        # Iterate over all nodes in the graph
        for node in self.network_graph.nodes():
            # Skip the root node
            if node == root:
                continue
            # Check if the in-degree of the node is at least 1
            if self.network_graph.in_degree(node) < 1:
                # print(f"Node {node} does not have any incoming arcs.")
                return False
        # print("All nodes except the root have at least one incoming arc.")
        return True
            
    def format_indian_number(self,num):
        num_str = str(num)
        if len(num_str) <= 3:
            return num_str
        else:
            # Split the number into the last three digits and the rest
            last_three = num_str[-3:]
            remaining = num_str[:-3]
            # Add commas every two digits in the remaining part
            remaining = ','.join([remaining[max(i - 2, 0):i] for i in range(len(remaining), 0, -2)][::-1])
            return remaining + ',' + last_three

    def fix_leaf_arc_flow(self):
        graph = nx.Graph()
        arc_set = self.ampl.getSet('arcs').to_list()  
        graph.add_edges_from(arc_set)
        D = self.ampl.getParameter('D').getValues().to_dict()  
        source = self.ampl.getSet('Source').to_list()[0]
        fixed_arcs = set()
        # print("\nPresolve the model for fixing the flow value in the leaf arcs")
        # print("Source:",self.ampl.getSet('Source').to_list())

        while True:
            leaf_nodes = [node for node in graph.nodes if graph.degree[node] == 1]
            # print("leaf_nodes:", leaf_nodes)
            if not leaf_nodes:  
                break

            for leaf in leaf_nodes:
                neighbor = next(graph.neighbors(leaf))
                if (neighbor, leaf) in arc_set:
                    edge = (neighbor, leaf)
                    if edge not in fixed_arcs:  
                        if leaf == source:
                            flow_value = D[leaf]
                            D[neighbor] = (D[neighbor]+flow_value)
                            source = neighbor
                        else:
                            flow_value = D[leaf]
                            D[neighbor] = D[neighbor] + flow_value
                        self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        # print(f"Fixing flow for arc {edge}: {flow_value}")
                        fixed_arcs.add(edge)  

                    graph.remove_node(leaf)
                else:
                    edge = (leaf, neighbor)
                    if edge not in fixed_arcs:  
                        if leaf == source:
                            flow_value = -D[leaf]
                            D[neighbor] = D[neighbor]-flow_value
                            source = neighbor
                            self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        elif neighbor == source:
                            flow_value = -D[leaf]
                            D[neighbor] = D[neighbor] - D[leaf] 
                            self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        else:
                            flow_value = -D[leaf]
                            D[neighbor] += -flow_value
                            self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        # print(f"Fixing flow for arc {edge}: {flow_value}")
                        fixed_arcs.add(edge)  
                    graph.remove_node(leaf)
        # print("All leaf arc flows have been fixed.")


    def is_cycle(self, graph, start_node, end_node, visited_copy, parent):
        visited_copy[start_node] = True
        # print(f"Is node {end_node} in cycle?")
        for neighbor in graph.neighbors(start_node):
            # print("visited",neighbor,visited_copy[neighbor])
            if not visited_copy[neighbor]:
                # print("neighbor of node", start_node, "is", neighbor)
                isCycle = self.is_cycle(graph, neighbor, end_node, visited_copy, start_node)
                if isCycle:
                    return True
            else:
                # print("parent:", parent)
                if parent != neighbor:
                    if end_node == neighbor:
                        # print(f"Node {end_node} is in cycle")
                        return True
        return False

    def presolve(self, graph, node, visited, parent, set_arc):
        visited_copy = visited.copy()
        # print(visited_copy)
        isCycle = self.is_cycle(graph, node, node, visited_copy, parent)
        # print(f"Is node {node} in cycle?",isCycle)
        visited[node] = True
        if isCycle:
            for neighbor in graph.neighbors(node):
                if parent!=neighbor:
                    set_arc.append((node,neighbor))
                    # print("Fix the arc", (node, neighbor))
            return set_arc
        else:
            for neighbor in graph.neighbors(node):
                if parent != neighbor:
                    set_arc.append((node,neighbor))
                    # print(set_arc)
                    # print("Fix the arc", (node, neighbor))
                    # print("neighbor:", neighbor)
                    self.presolve(graph, neighbor, visited, node, set_arc)
        return set_arc

    def fix_arc_set(self):
        graph = nx.Graph()
        arc_set = self.ampl.getSet('arcs').to_list()
        graph.add_edges_from(arc_set)
        visited = {node: False for node in graph.nodes()}
        source = self.ampl.getSet('Source').to_list()[0]
        set_arc = []
        # print("\nPresolve the model for fixing the arc direction")
        set_ = self.presolve(graph, source, visited, -1, set_arc)
        # print("fixed arc direction:",set_, "\n") 
        return set_


    def update_initial_points(self,l_solution, q_solution, h_solution):
        for (i, j, k), val in l_solution.items():
            self.ampl.eval(f'let l[{i},{j},{k}] := {val};')
        for (i, j), val in q_solution.items():
            self.ampl.eval(f'let q[{i},{j}] := {val};')
        for i, val in h_solution.items():
            self.ampl.eval(f'let h[{i}] := {val};')

    def update_initial_points1(self, l_solution, q_solution, h_solution, t_solution,all_duals, inarc):
        for (i, j, k), val in l_solution.items():
            self.ampl.eval(f'let l[{i},{j},{k}] := {val};')
        
        edge_list_network = self.network_graph.edges
        
        for i, val in h_solution.items():
            self.ampl.eval(f'let h[{i}] := {val};')
        
        for (i, j), val in q_solution.items():
            edge = (i, j) 
            if edge in edge_list_network:
                if (i,j) not in inarc:
                    # print(f"self.ampl.eval(let q[{i},{j}] := {val};)")
                    self.ampl.eval(f"let q[{i},{j}] := {val} ;")
                else:
                    # print(f"self.ampl.eval(let q[{i},{j}] := {-val};)")
                    self.ampl.eval(f"let q[{i},{j}] := {val} ;")
            else:
                if (j,i) not in inarc:
                    # print(f"self.ampl.eval(let q[{i},{j}] := {val};)")
                    self.ampl.eval(f"let q[{i},{j}] := {val};")
                else:
                    # print(f"self.ampl.eval(let q[{j},{i}] := {-val};)")
                    self.ampl.eval(f"let q[{j},{i}] := {val} ;")
        for i, val in t_solution.items():
            self.ampl.eval(f'let t[{i}] := {val};')
            
        current_duals = {}
        for con_name, val in self.ampl.get_constraints():
            dual_values = val.get_values()
            current_duals[con_name] = dual_values

        # Initialize dual values for all constraints
        # for con_name, dual_values in all_duals.items():
            # if con_name in current_duals:
                # Initialize dual values for each constraint
                # self.ampl.get_constraint(con_name).set_values(dual_values)
            # else:
            #     print(f"Skipping initialization for constraint: {con_name} (not in current model)")
    
    def update_initial_points_with_perturbation(self, l_solution, q_solution, h_solution,all_duals, inarc, delta=0.1):
        edge_list_network = self.network_graph.edges
        L = self.ampl.getParameter('L').getValues().to_dict()
        # Perturb l values
        for (i, j, k), val in l_solution.items():
            if (i,j) not in inarc:
                if val>= 1e-5:
                    perturbation = random.gauss(0, 1)
                    new_val = val + perturbation
                    # if val >= 1e-5:
                    self.ampl.eval(f'let l[{i},{j},{k}] := {new_val};')
                else:
                    self.ampl.eval(f'let l[{i},{j},{k}] := {0};')
            else:
                if val>= 1e-5:
                    perturbation = random.gauss(0, 1)
                    new_val = val + perturbation
                    self.ampl.eval(f'let l[{i},{j},{k}] := {new_val};')
                else:
                    self.ampl.eval(f'let l[{i},{j},{k}] := {0};')
                    
        # Perturb h values
        for i, val in h_solution.items():
            perturbation = random.gauss(0, 1)
            new_val = val + perturbation
            self.ampl.eval(f'let h[{i}] := {new_val};')

        # Modify q values based on heuristic
        for (i, j), val in q_solution.items():
            edge = (i, j)
            perturbation = random.gauss(0, 1)
            if edge in edge_list_network:
                if (i, j) not in inarc:
                    self.ampl.eval(f"let q[{i},{j}] := {val + perturbation};")
                else:
                    self.ampl.eval(f"let q[{i},{j}] := {(val + perturbation)};")
            else:
                if (j, i) not in inarc:
                    self.ampl.eval(f"let q[{i},{j}] := {val + perturbation};")
                else:
                    self.ampl.eval(f"let q[{j},{i}] := {(val + perturbation)};")
        
        current_duals = {}
        for con_name, val in self.ampl.get_constraints():
            dual_values = val.get_values()
            current_duals[con_name] = dual_values

        # Initialize dual values for all constraints
        for con_name, dual_values in all_duals.items():
            if con_name in current_duals:
                # Initialize dual values for each constraint
                self.ampl.get_constraint(con_name).set_values(dual_values)

    def acyclic_arcs(self):
        network_graph = self.best_acyclic_flow
        indegree_2_or_more = [node for node, indeg in network_graph.in_degree() if indeg >= 2]
        acyclic_arc = set()
        for node in indegree_2_or_more:
            # print("Node:", node,"in_degree:", self.network_graph.in_degree(node))
            for edge in list(network_graph.in_edges(node)):
                (u, v) = edge
                if (u,v) not in self.super_source_out_arc :
                    network_graph.remove_edge(u,v)
                    network_graph.add_edge(v,u)
                    acy_check = nx.is_directed_acyclic_graph(network_graph)
                    in_arc_check = self.check_incoming_arcs()
                    # print("Acyclic", acy_check and in_arc_check)
                    if acy_check and in_arc_check:
                        acyclic_arc.add((u,v))

                    network_graph.remove_edge(v, u)
                    network_graph.add_edge(u, v) 
        return acyclic_arc


    def process_arc(self, arc):
        u, v = arc
        # Reverse the arc
        self.network_graph.remove_edge(u, v)
        self.network_graph.add_edge(v, u)
        acy_check = nx.is_directed_acyclic_graph(self.network_graph)
        in_arc_check = self.check_incoming_arcs()

        if acy_check and in_arc_check:
            self.load_model()
            self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
            self.fix_leaf_arc_flow()
            self.update_initial_points1(self.l, self.q, self.h, self.t, self.all_duals, self.inarc)

            if (u, v) in self.arcs:
                self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{u}, {v}]<=0;")
            else:
                self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{v}, {u}]>=0;")

            self.solve()

            if self.solve_result == "solved":
                l = self.ampl.getVariable('l').getValues().to_dict()
                q = self.ampl.getVariable('q').getValues().to_dict()
                h = self.ampl.getVariable('h').getValues().to_dict()
                t = self.ampl.getVariable('t').getValues().to_dict()

                return {
                    "arc": arc,
                    "acyclic": True,
                    "total_cost": self.total_cost,
                    "solve_result": self.solve_result,
                    "solve_time": round(self.ampl.get_value('_solve_elapsed_time'), 2),
                    "improved": self.total_cost < self.current_cost,
                    "l": l,
                    "q": q,
                    "h": h,
                    "t": t,
                    "acyclic_flow": self.network_graph.copy()
                }

        # Revert the arc if not successful
        self.network_graph.remove_edge(v, u)
        self.network_graph.add_edge(u, v)
        return {"arc": arc, "acyclic": False}

    def run_parallel1(self):
        improved = False
        best_result = None
        
        while True:
            # Use multiprocessing to process arcs in parallel
            with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
                results = pool.map(self.process_arc, self.acyclic_arc_set)

            # Filter valid solutions and find the best result
            valid_results = [r for r in results if r["acyclic"] and "total_cost" in r]
            if valid_results:
                best_result = min(valid_results, key=lambda r: r["total_cost"])

                if best_result["total_cost"] < self.current_cost:
                    self.current_cost = best_result["total_cost"]
                    self.best_acyclic_flow = best_result["acyclic_flow"]
                    self.l = best_result["l"]
                    self.q = best_result["q"]
                    self.h = best_result["h"]
                    self.t = best_result["t"]
                    improved = True
                    
                    print(f"New best solution found! Total cost: {self.current_cost}")
                else:
                    improved = False
            else:
                print("No improved solutions found in this iteration.")
            
            # If no improvement, stop the loop
            if not improved:
                break

            # Update the graph and prepare for the next iteration
            self.network_graph = self.best_acyclic_flow.copy()
            self.acyclic_arc_set = self.acyclic_arcs()  # Recompute the arcs to process
        
        # Return the best result
        return best_result, improved
    
    def iterate_arc(self, iteration, improved, current_cost, best_arc):
        improved = False
        self.network_graph = self.best_acyclic_flow.copy()
        
        # print("Acyclic network arcs direction: ",self.network_graph.edges())
        # print("Fixed arc set:",self.super_source_out_arc)
        
        BEST_ARC = []
        # BEST_ARC.append(best_arc)
        
        self.inarc = []
        for node in self.indegree_2_or_more:
            for (u, v) in list(self.network_graph.in_edges(node)):
                if (u, v) in self.arcs:
                    self.inarc.append((u,v))
                else:
                    self.inarc.append((v,u))
        print("inarc:", self.inarc)
        inarc_ = self.inarc
        
        inarc_set = []
        for (i, j) in self.inarc:
            if (i, j) in self.arcs:
                inarc_set.append(f"({i},{j})")
            else:
                inarc_set.append(f"({j},{i})")
        
        # Convert the list into a string compatible with AMPL
        inarc_set = ", ".join(inarc_set)
        
        # for node in self.indegree_2_or_more:
        #     print("Node:", node,"in_degree:", self.network_graph.in_degree(node))
        #     for u,v in list(self.network_graph.in_edges(node)):
        for (u,v) in self.acyclic_arc_set:
            self.network_graph.remove_edge(u,v)
            self.network_graph.add_edge(v,u)
            acy_check = nx.is_directed_acyclic_graph(self.network_graph)
            in_arc_check = self.check_incoming_arcs()
            # print("Acyclic", acy_check and in_arc_check)
            if acy_check and in_arc_check:
                #l_sol, q_sol, h_sol = self.generate_initial_points()
                self.load_model()
                # self.ampl.eval(f"set inarc := {{{inarc_set}}};")
                # self.ampl.eval(f"set indegree_node := {{{set(self.indegree_2_or_more)}}};")
                self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
                self.fix_leaf_arc_flow()
                self.update_initial_points1(self.l, self.q, self.h, self.t, self.all_duals, self.inarc)
                # self.update_model()
                self.ampl.eval(f"param Q_max = sum{{k in nodes diff Source }} D[k];")   
                # self.ampl.eval("subject to con7{(i,j) in arcs}: -sum{k in nodes diff Source} D[k] <= q[i,j];")
                # self.ampl.eval("subject to con8{(i,j) in arcs}: q[i,j] <= sum{k in nodes diff Source} D[k];")
                
                if (u,v) in self.arcs:
                    self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{u}, {v}]<=0;")
                    self.ampl.eval(f"s.t. flow_bound_left_{u}_{v}: -Q_max <= q[{u}, {v}];")
                else:
                    self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{v}, {u}]>=0;")
                    self.ampl.eval(f"s.t. flow_bound_right_{u}_{v}: q[{v}, {u}] <= Q_max;")
                self.solve()
                
                l = self.ampl.getVariable('l').getValues().to_dict()
                q = self.ampl.getVariable('q').getValues().to_dict()
                h = self.ampl.getVariable('h').getValues().to_dict()
                t = self.ampl.getVariable('t').getValues().to_dict()
                
                if self.solve_result == "solved":
                    
                    if self.total_cost < current_cost:
                        # print("Arc", (u,v),"Acyclic:", acy_check and in_arc_check, "Best optimal: ", '{:,}'.format(round(current_cost)), "New optimal: ", '{:,}'.format(round(self.total_cost)), "Solve_time:", self.ampl.get_value('_solve_elapsed_time'), "Solve_result: ", self.solve_result, "Improved: Yes")
                        
                        print(f"Arc: {(u,v)}",
                              f"Acyclic: {acy_check and in_arc_check}",
                              f"Best_optimal: {self.format_indian_number(round(current_cost))}", 
                              f"New_optimal: {self.format_indian_number(round(self.total_cost))}", 
                              f"Solve_time: {round(self.ampl.get_value('_solve_elapsed_time'), 2)}", 
                              f"Solve_result: {self.solve_result}", "Improved: Yes", 
                              f"Time_count: {round(time.time() - self.start_time, 2)}")

                        current_cost = self.total_cost
                        improved = True
                        self.network_graph = self.generate_random_acyclic_from_solution(q)
                        
                        self.best_acyclic_flow = self.network_graph.copy()
                        self.indegree_2_or_more = [node for node, indeg in self.best_acyclic_flow.in_degree() if indeg >= 2]
                        # print("indegree_2_or_more:", self.indegree_2_or_more)
                        
                        best_arc = (v,u)
                        self.l = l 
                        self.q = q
                        self.h = h 
                        self.t = t

                        self.all_duals = {}
                        for con_name, val in self.ampl.get_constraints():
                            # Get dual values for each constraint
                            dual_values = val.getValues()
                            self.all_duals[con_name] = dual_values

                        self.inarc = []
                        for node in self.indegree_2_or_more:
                            for (u, v) in self.best_acyclic_flow.in_edges(node):
                                if (u, v) in self.arcs:
                                    if (u,v) not in self.super_source_out_arc:
                                        self.inarc.append((u,v))
                                else:
                                    self.inarc.append((v,u))
                        # self.plot_graph(self.super_source_out_arc, current_cost, iteration, self.q, self.h, self.D, self.l, self.C)
                        self.acyclic_arc_set = self.acyclic_arcs()
                        
                        break
                    else:
                        # print("Arc", (u,v),"Acyclic:", acy_check and in_arc_check, "Best optimal: ", '{:,}'.format(round(current_cost)), "New optimal: ", '{:,}'.format(round(self.total_cost)), "Solve_time:", self.ampl.get_value('_solve_elapsed_time'), "Solve_result: ", self.solve_result, "Improved: No")

                        print(f"Arc: {(u,v)}",
                              f"Acyclic: {acy_check and in_arc_check}",
                              f"Best_optimal: {self.format_indian_number(round(current_cost))}", 
                              f"New_optimal: {self.format_indian_number(round(self.total_cost))}", 
                              f"Solve_time: {round(self.ampl.get_value('_solve_elapsed_time'), 2)}", 
                              f"Solve_result: {self.solve_result}", "Improved: No ", 
                              f"Time_count: {round(time.time() - self.start_time, 2)}")
                        
                        self.network_graph.remove_edge(v, u)
                        self.network_graph.add_edge(u, v)  
                else:
                    # print("Arc", (u,v),"Acyclic:",acy_check and in_arc_check, "Best optimal: ", '{:,}'.format(round(current_cost)), "New optimal: ", '{:,}'.format(round(self.total_cost)),  "Solve_time:", self.ampl.get_value('_solve_elapsed_time'), "Solve_result: ", self.solve_result)
                    print(f"Arc: {(u,v)}",
                          f"Acyclic: {acy_check and in_arc_check}",
                          f"Best_optimal: {self.format_indian_number(round(current_cost))}", 
                          f"New_optimal: {self.format_indian_number(round(self.total_cost))}", 
                          f"Solve_time: {round(self.ampl.get_value('_solve_elapsed_time'), 2)}", 
                          f"Solve_result: {self.solve_result}", "Improved: No ", 
                          f"Time_count: {round(time.time() - self.start_time, 2)}")
                    self.network_graph.remove_edge(v, u)
                    self.network_graph.add_edge(u, v)                         
            else:
                print(f"Arc: {(u,v)}",
                      f"Acyclic: {acy_check and in_arc_check}")
                
                self.network_graph.remove_edge(v, u)
                self.network_graph.add_edge(u, v)                      
            print(" ")
            if improved:
                break
        return self.best_acyclic_flow, improved, current_cost, self.l, self.q, self.h, best_arc

    
    @staticmethod
    def worker(data, stop_flag):
        # (arc, network_graph) = data
        # verbose=False
        # with contextlib.redirect_stdout(None):
        (arc, network_graph,current_cost, best_acyclic_flow, data_number, l, q, h) = data
        
        u, v = arc
        
        # print(arc)
        # print(ampl_config)
        data_list = [
            "d1_Sample_input_cycle_twoloop",
            "d2_Sample_input_cycle_hanoi",
            "d3_Sample_input_double_hanoi",
            "d4_Sample_input_triple_hanoi",
            "d5_Taichung_input",
            "d6_HG_SP_1_4",
            "d7_HG_SP_2_3",
            "d8_HG_SP_3_4",
            "d9_HG_SP_4_2",
            "d10_HG_SP_5_5",
            "d11_HG_SP_6_3",
            "d12",
            "d13",
            "d14_NewYork",
            "d15_foss_poly_0",
            "d16_foss_iron",
            "d17_foss_poly_1",
            "d18_pescara",
            "d19_modena"
        ]
        # Reverse the arc in a copy of the graph
        network_graph.remove_edge(u, v)
        network_graph.add_edge(v, u)
        
        # Create a new AMPL instance and initialize
        ampl = AMPL()
        ampl.read("../m1Basic.mod")
        ampl.read_data(f"/home/nitishdumoliya/waterNetwork/data/{data_list[data_number]}.dat")
        
        nodes = ampl.getSet('nodes')
        source = ampl.getSet('Source')
        arcs = ampl.getSet('arcs')
        pipes = ampl.getSet('pipes')
        
        # print(nodes)
        # print(arcs)
        # print(ampl_config["current_cost"])
        
        # L = ampl.getParameter('L').to_dict()
        # D = ampl.getParameter('D').to_dict()
        # C = ampl.getParameter('C').to_dict()
        # P = ampl.getParameter('P').to_dict()
        # R = ampl.getParameter('R').to_dict()
        # E = ampl.getParameter('E').to_dict()
        # d = ampl.getParameter('d').to_dict()

        delta = 0.1
        p = 1.852
        omega = 10.67
        
        ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
        # ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}1.2654*l[i,j,k]*(d[k])^1.327;")
        # self.fix_leaf_arc_flow()
        # self.update_initial_points1(ampl_config["l"], ampl_config["q"], ampl_config["h"], ampl_config["t"], ampl_config["all_duals"], ampl_config["inarc"])
        
        for (i, j, k), val in l.items():
            ampl.eval(f'let l[{i},{j},{k}] := {val};')
        for (i, j), val in q.items():
            ampl.eval(f'let q[{i},{j}] := {val};')
        for i, val in h.items():
            ampl.eval(f'let h[{i}] := {val};')
            
        # Add constraints for the arc direction
        if (u, v) in arcs:
            ampl.eval(f"s.t. flow_direction{u}_{v}: q[{u}, {v}]<=0;")
        else:
            ampl.eval(f"s.t. flow_direction{u}_{v}: q[{v}, {u}]>=0;")
            
        # ampl.eval("subject to con7{(i,j) in arcs}: -sum{k in nodes diff Source} D[k] <= q[i,j];")
        # ampl.eval("subject to con8{(i,j) in arcs}: q[i,j] <= sum{k in nodes diff Source} D[k];")
       
        ampl.option["solver"] = "ipopt"
        ampl.option["snopt_options"] = "major_iterations_limit=1000"
        # ampl.set_option("ipopt_options", "outlev = 0 expect_infeasible_problem = yes bound_push = 0.01 bound_frac = 0.01 warm_start_init_point = yes max_iter = 400")   #max_iter = 1000
        ampl.set_option("ipopt_options", """outlev = 0 expect_infeasible_problem = yes bound_push = 0.01 bound_frac = 0.01 warm_start_init_point = yes mu_strategy = adaptive mu_oracle = loqo max_iter=500 """)   #max_iter = 1000 mu_strategy = adaptive mu_oracle = loqo max_soc = 4
        ampl.option["presolve_eps"] = "6.82e-14"
        ampl.option['presolve'] = 1

        def format_indian_number(num):
            num_str = str(num)
            if len(num_str) <= 3:
                return num_str
            else:
                # Split the number into the last three digits and the rest
                last_three = num_str[-3:]
                remaining = num_str[:-3]
                # Add commas every two digits in the remaining part
                remaining = ','.join([remaining[max(i - 2, 0):i] for i in range(len(remaining), 0, -2)][::-1])
                return remaining + ',' + last_three
                
        
        # Solve the model
        # print(stop_flag.is_set())
        if not stop_flag.is_set():
            
            with contextlib.redirect_stdout(None):
                ampl.solve()
            
            print(f"Arc: {(u,v)}",
              f"Acyclic: True",
              f"Best_optimal: {format_indian_number(round(current_cost))}", 
              f"New_optimal: {format_indian_number(round(ampl.get_objective("total_cost").value()))}", 
              f"Solve_time: {round(ampl.get_value('_solve_elapsed_time'), 2)}", 
              f"Solve_result: {ampl.get_value("solve_result")}",
              f"Improved: {ampl.get_objective("total_cost").value() < current_cost}")
            
            
            
            # indegree_2_or_more = [node for node, indeg in network_graph.in_degree() if indeg >= 2]
            # super_source_out_arc = self.fix_arc_set()
            # all_duals = {}
            # for con_name, val in ampl.get_constraints():
            #     # Get dual values for each constraint
            #     dual_values = val.getValues()
            #     all_duals[con_name] = dual_values
    
            # inarc = []
            # for node in indegree_2_or_more:
            #     for (u, v) in network_graph.in_edges(node):
            #         if (u, v) in arcs:
            #             if (u,v) not in self.super_source_out_arc:
            #                 inarc.append((u,v))
            #         else:
            #             inarc.append((v,u))
                    

            # # Check if the stop flag is already set
            # if stop_flag.is_set():
            #     print(f"Worker for arc {arc} stopped as condition was met.")
            #     return {
            #         "total_cost": ampl.get_objective("total_cost").value(),
            #         "solve_result": ampl.get_value("solve_result"),
            #         "solve_time": round(ampl.get_value('_solve_elapsed_time'), 2),
            #         "improved": ampl.get_objective("total_cost").value() < current_cost,
            #         "acyclic_flow": network_graph.copy(), 
            #         "l": l,
            #         "q": q,
            #         "h": h,
            #     }
            
            # Check solution status
            if ampl.get_value("solve_result") == "solved" or ampl.get_value("solve_result") == "solved?":
                l = ampl.getVariable("l").getValues().to_dict()
                q = ampl.getVariable("q").getValues().to_dict()
                h = ampl.getVariable("h").getValues().to_dict()
                
                if ampl.get_objective("total_cost").value()<current_cost:
                    print(f"Condition met for arc {arc}, stopping parallel run.")
                    stop_flag.set()  # Set the stop flag
    
                return {
                    "arc":(u,v),
                    "total_cost": ampl.get_objective("total_cost").value(),
                    "solve_result": ampl.get_value("solve_result"),
                    "solve_time": round(ampl.get_value('_solve_elapsed_time'), 3),
                    "improved": ampl.get_objective("total_cost").value() < current_cost,
                    "acyclic_flow": network_graph.copy(), 
                    "l": l,
                    "q": q,
                    "h": h,
                    "nlp_run":True
                }
            else:
                return {
                    "arc":(u,v),
                    "total_cost": ampl.get_objective("total_cost").value(),
                    "solve_result": ampl.get_value("solve_result"),
                    "solve_time": round(ampl.get_value('_solve_elapsed_time'), 3),
                    "improved": ampl.get_objective("total_cost").value() < current_cost,
                    "acyclic_flow": network_graph.copy(), 
                    "l": l,
                    "q": q,
                    "h": h,
                    "nlp_run":True
                }
        return {
                "arc":(u,v),
                "total_cost": 'inf',
                "solve_result": "not_solved",
                "solve_time": 0,
                "improved": False,
                "acyclic_flow": None, 
                "l": None,
                "q": None,
                "h": None,
                "nlp_run":False
            }
                
    def run_parallel(self, network_graph, ampl_config):
        best_result = None
        improved = False
        
        
        current_cost =  ampl_config["current_cost"]
        best_acyclic_flow =  ampl_config["best_acyclic_flow"]
        l =  ampl_config["l"]
        q =  ampl_config["q"]
        h =  ampl_config["h"]
        
        # t =  ampl_config["t"]
        # all_duals =  ampl_config["all_duals"]
        # inarc =  ampl_config["inarc"]
        
        # Prepare the data for each worker
        # acyclic_arc_set = list(network_graph.edges)
        # data = [(arc, network_graph.copy(), ampl_config) for arc in self.acyclic_arc_set]
        
        # global stop_flag, pool
        
        # Initialize the shared stop flag
        # stop_flag = multiprocessing.Value('i', 0)  # Shared flag to stop workers
        
        # def check_result(result):
        #     """
        #     Callback function to check the worker's result.
        #     If the condition is met, set the stop flag to True.
        #     """
        #     global stop_flag
        #     if result["total_cost"] < current_cost:
        #         stop_flag.value = True  # Stop further processing
        #         print(f"Condition met. Stopping parallel computation. Total cost: {result['total_cost']}")
        #         pool.terminate()  # Terminate the pool immediately
        
        data = [(arc, network_graph.copy(),current_cost, best_acyclic_flow, self.data_number, l, q, h ) for arc in self.sorted_delta_arc]
        
        # print(data)
        # Run workers in parallel
        # with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        #     results = pool.map(self.worker, data)
        
        # nproc = os.cpu_count()
        # print(nproc, "processors available")
        # print()
        # pool = multiprocessing.Pool(8)
        # results = pool.map(self.worker, data)
        # Close the pool and wait for all tasks to complete
        # pool.close()
        # pool.join()
        
        # try:
        #     results = pool.starmap(self.worker, [(d, stop_flag) for d in data] )
        # finally:
        #     pool.close()
        #     pool.join()
        
        # Use a Manager to create a shared stop_flag
        # with multiprocessing.Manager() as manager:
        #     stop_flag = manager.Event()  # Shared event object
    
        #     # Create the pool of workers
        #     with multiprocessing.Pool(processes=8) as pool:
        #         try:
        #             results = pool.starmap(self.worker, [(d, stop_flag) for d in data])
        #         finally:
        #             pool.close()
        #             pool.join()
    
        # with multiprocessing.Manager() as manager:
        #     stop_flag = manager.Event()  # Shared event object
    
        #     with multiprocessing.Pool(processes=4) as pool:
        #         results = []
    
        #         # Use `imap_unordered()` instead of `starmap()` starmap_async
        #         for result in pool.starmap_async(self.worker, [(d, stop_flag) for d in data]):
        #             if stop_flag.is_set():  
        #                 pool.terminate()  # Kill all remaining processes
        #                 break  # Exit the loop
                
        #         pool.close()
        #         pool.join()
    
        #         print("Parallel execution stopped.")

        with multiprocessing.Manager() as manager:
            stop_flag = manager.Event()  # Shared event object

            with multiprocessing.Pool(processes=3) as pool:
                try:
                    # results = pool.starmap_async(self.worker, [(d, stop_flag) for d in data])  #apply_async, starmap_async
                    results = [pool.apply_async(self.worker, args=(d, stop_flag)) for d in data]
                    
                    # Monitor if stop_flag is set
                    # while not results.ready():
                    #     if stop_flag.is_set():
                    #         pool.terminate()  # Stop all processes
                    #         break
                    #     time.sleep(0.5)  # Avoid busy-waiting
                    # results = results.get()
                    
                    while not all(res.ready() for res in results):
                        if stop_flag.is_set():
                            pool.terminate()
                            break
                        time.sleep(0.5)
                    
                    results = [res.get() for res in results if res.ready()]  # Retrieve only ready results
                    
                    pool.close()
                    pool.join()

                    print("Parallel execution stopped.")
                
                except Exception as e:
                    print(f"Error: {e}")
                    pool.terminate()
                    pool.join()
        
        visited_nodes = [r["arc"][0] for r in results if r["solve_result"]!="not_solved"]
        self.visited_nodes.extend(visited_nodes)
        print("visited_nodes:",self.visited_nodes)

        visited_arcs = [r["arc"] for r in results if r["solve_result"] != "not_solved"]
        self.visited_arcs.extend(visited_arcs)
        
        self.number_of_nlp += len(visited_nodes)
        # Filter valid results
        valid_results = [r for r in results if r["solve_result"] == "solved" or r["solve_result"] == "solved?"]
        # nlp_list = [r for r in results if r["nlp_run"] == True]
        # self.number_of_nlp += len(nlp_list)
        
        # solve_time = 0
        # for i in range(len(nlp_list)):
        #     solve_time += nlp_list[i]["solve_time"] 
        
        # self.solver_time +=  solve_time
        
        if valid_results:
            # Find the best result
            best_result = min(valid_results, key=lambda r: r["total_cost"])
            
            # print(best_result)
            
            if best_result["total_cost"] < ampl_config["current_cost"]:
                ampl_config["current_cost"] = best_result["total_cost"]
                ampl_config["best_acyclic_flow"] = best_result["acyclic_flow"]
                ampl_config["l"] = best_result["l"]
                ampl_config["q"] = best_result["q"]
                ampl_config["h"] = best_result["h"]
                # ampl_config["t"] = best_result["t"]
                # ampl_config["all_duals"]: best_result["all_duals"]
                # ampl_config["inarc"]: best_result["inarc"]
                improved = True
                ampl_config["best_acyclic_flow"] = best_result["acyclic_flow"].copy()
                network_graph = best_result["acyclic_flow"].copy()
                arc = best_result["arc"]
                print("arc", arc)
                print(f"New best solution found! Total cost: {ampl_config['current_cost']}", f"Time_count: {round(time.time() - self.start_time, 2)}")
            else:
                improved = False
                arc = best_result["arc"]
                # print("arc", arc)
                print("No improved solutions found in this iteration.")
        # else:
        #     print("No improved solutions found in this iteration.")

        # # Stop if no improvement
        # if not improved:
        #     break

        # Update the graph for the next iteration
        
        return network_graph, ampl_config, improved, arc

    def iterate_acyclic_flows(self):
        """Iterate to find improved acyclic flows by attempting arc reversals while maintaining acyclicity."""
        improved = True 
        
        self.best_acyclic_flow = self.network_graph.copy()
        
        # self.indegree_2_or_more = [node for node, indeg in self.best_acyclic_flow.in_degree() if indeg >= 2]
        # print("indegree_2_or_more:", self.indegree_2_or_more)
        
        if self.solve_result == "solved":
            current_cost = self.total_cost
            self.l = self.ampl.getVariable('l').getValues().to_dict()
            self.q = self.ampl.getVariable('q').getValues().to_dict()
            self.h = self.ampl.getVariable('h').getValues().to_dict()
            self.t = self.ampl.getVariable('t').getValues().to_dict()
            
            self.all_duals = {}
            for con_name, val in self.ampl.get_constraints():
                # Get dual values for each constraint
                dual_values = val.getValues()
                self.all_duals[con_name] = dual_values
        
        elif self.solve_result != "solved":
            current_cost = 10e+14
            self.l = None
            self.q = None
            self.h = None
        
        iteration = 1
        best_arc = None
        
        self.inarc = []
        for node in self.indegree_2_or_more:
            for (u, v) in self.best_acyclic_flow.in_edges(node):
                if (u, v) in self.arcs:
                    if (u,v) not in self.super_source_out_arc:
                        self.inarc.append((u,v))
                else:
                    self.inarc.append((v,u))
        
        # self.plot_graph(self.super_source_out_arc, current_cost, 0, self.q, self.h, self.D, (0,0), self.l, self.C)
        self.network_graph = self.best_acyclic_flow.copy()
        self.acyclic_arc_set = self.acyclic_arcs()
        sum_arc = {}       
        delta_arc = {}
        for node in self.indegree_2_or_more:
            sum_arc[node]=0
            for (u,v) in self.best_acyclic_flow.in_edges(node):
                if (u,v) in self.arcs:
                    sum_arc[node] += abs(self.q[u,v])
                else:
                    sum_arc[node] += abs(self.q[v,u])
            
            for (u,v) in self.best_acyclic_flow.in_edges(node):
                # delta_arc[u,v] = 0
                if (u,v) in self.arcs:
                    if (u,v) in self.acyclic_arc_set:
                        # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[u,v]))
                        delta_arc[u,v] = self.D[node] - sum_arc[node] + abs(self.q[u,v])
                else:
                    if (u,v) in self.acyclic_arc_set:
                        # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[v,u]))
                        delta_arc[u, v] = self.D[node] - sum_arc[node] + abs(self.q[v,u])
                    
        sorted_dict = dict(sorted(delta_arc.items(), key=lambda item: abs(item[1]), reverse=True))

        
        self.sorted_nodes = sorted(self.indegree_2_or_more, key=lambda node: self.D[node], reverse=True)
        self.visited_nodes = []
        self.visited_arcs = []
        # if self.visited_nodes:
        #     self.sorted_nodes = [item for item in self.sorted_nodes if item not in self.visited_nodes]
        print("sorted_nodes", self.sorted_nodes)
        final_sorted_arcs = []
        
        for node in   self.sorted_nodes:
            node_arcs = [(u, v) for (u, v) in self.best_acyclic_flow.in_edges(node) if (u, v) in delta_arc]
            # node_arcs_sorted = sorted(node_arcs, key=lambda arc: abs(delta_arc[arc]), reverse=True)
            # final_sorted_arcs.extend(node_arcs_sorted)
            if node_arcs:  # Ensure there are incoming arcs before picking the max
                max_arc = max(node_arcs, key=lambda arc: abs(delta_arc[arc]))  # Pick the arc with max absolute delta_arc
                final_sorted_arcs.append(max_arc)
                # if self.best_acyclic_flow.in_degree(node) >=3:
                #     min_arc = min(node_arcs, key=lambda arc: abs(delta_arc[arc]))  # Pick the arc with max absolute delta_arc
                #     final_sorted_arcs.append(min_arc)
                
                
        # Store final sorted arc order
        # self.sorted_delta_arc = final_sorted_arcs
        # print(sorted_dict)
        self.sorted_delta_arc = list(sorted_dict.keys())
        # print("sorted_delta_arc:",self.sorted_delta_arc)
        # ampl_config = {
        #     "current_cost": current_cost,
        #     "best_acyclic_flow": self.network_graph,
        #      "l": self.l,
        #      "q": self.q,
        #      "h": self.h,
        #      "t": self.t,
        #      "all_duals": self.all_duals,
        #     "inarc": self.inarc
        # }
        
        ampl_config = {
            "current_cost": current_cost,
            "best_acyclic_flow": self.network_graph,
            "l": self.l,
            "q": self.q,
            "h": self.h,
        }
        
        while improved and self.sorted_delta_arc:
            print("\n******************************************************************************************************************************************")
            print("Iteration :",iteration, "\n")
            # print(self.sorted_delta_arc)
            print("sorted_delta_arc:",self.sorted_delta_arc)
            # self.best_acyclic_flow, improved, current_cost, self.l, self.q, self.h, best_arc = self.iterate_arc(iteration, improved, current_cost, best_arc)
            self.network_graph, ampl_config, improved, arc = self.run_parallel(self.network_graph, ampl_config)
            self.best_acyclic_flow = self.network_graph
            self.acyclic_arc_set = self.acyclic_arcs()
            # print("acyclic_arc_set:",acyclic_arc_set)
            sum_arc = {}       
            delta_arc = {}
            self.indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
            # print("self.indegree_2_or_more", self.indegree_2_or_more)
            for node in self.indegree_2_or_more:
                sum_arc[node]=0
                for (u,v) in self.best_acyclic_flow.in_edges(node):
                    if (u,v) in self.arcs:
                        sum_arc[node] += abs(self.q[u,v])
                    else:
                        sum_arc[node] += abs(self.q[v,u])
                
                for (u,v) in self.best_acyclic_flow.in_edges(node):
                    # delta_arc[u,v] = 0
                    if (u,v) in self.arcs:
                        if (u,v) in self.acyclic_arc_set:
                            # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[u,v]))
                            delta_arc[u,v] = self.D[node] - sum_arc[node] + abs(self.q[u,v])
                    else:
                        if (u,v) in self.acyclic_arc_set:
                            # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[v,u]))
                            delta_arc[u, v] = self.D[node] - sum_arc[node] + abs(self.q[v,u])
                        

            self.sorted_nodes = sorted(self.indegree_2_or_more, key=lambda node: self.D[node], reverse=True)
            # self.visited_nodes = []
            if self.visited_nodes:
                self.sorted_nodes = [item for item in self.sorted_nodes if item not in self.visited_nodes]
            # print("\nsorted_nodes", self.sorted_nodes, "\n")
            final_sorted_arcs = []
            
            for node in   self.sorted_nodes:
                node_arcs = [(u, v) for (u, v) in self.best_acyclic_flow.in_edges(node) if (u, v) in delta_arc]
                # node_arcs_sorted = sorted(node_arcs, key=lambda arc: abs(delta_arc[arc]), reverse=True)
                # final_sorted_arcs.extend(node_arcs_sorted)
                if node_arcs:  # Ensure there are incoming arcs before picking the max
                    max_arc = max(node_arcs, key=lambda arc: abs(delta_arc[arc]))  # Pick the arc with max absolute delta_arc
                    final_sorted_arcs.append(max_arc)
            # print(sorted_dict)
            # self.sorted_delta_arc = final_sorted_arcs
            
            # if improved:
            #     (i,j) = arc
            #     self.sorted_delta_arc.remove((i,j))
            
            sorted_dict = dict(sorted(delta_arc.items(), key=lambda item: abs(item[1]), reverse=True))
            self.sorted_delta_arc = list(sorted_dict.keys())
            if self.visited_arcs:
                self.sorted_delta_arc = [item for item in self.sorted_delta_arc if item not in self.visited_arcs]
            # print("\nsorted_nodes", self.sorted_nodes, "\n")
            
            # print("sorted_delta_arc:",self.sorted_delta_arc, "\n")
            # print(self.acyclic_arc_set)
            
            # print("Current best acyclic network:")
            # plt.figure(figsize=(10, 8))
            # nx.draw_spectral(best_acyclic_flow, with_labels=True)
            # plt.show()
            # super_source_out_arc = self.fix_arc()
            # super_source_out_arc.append(best_arc)
            iteration += 1
            # print(f"Current best solution: {current_cost}")
            # print(" ")
        
        print("\n*********************************************************Final best results***************************************************************")
        print("Water Network:", self.data_number)
        print(f"Final best objective: {ampl_config["current_cost"]}")

        print("Number of nlp problem solved:", self.number_of_nlp)
        print("Total number of iteration to solve the problem:", iteration-1)
        # print("length of the arcs: ", l, "\n")
        # print("flow in the arcs: ", q, "\n")
        # print("head value at node: ", h, "\n")
        # self.network_graph = best_acyclic_flow
        # self.load_model()
        # self.update_model()
        # self.multi_solve(current_cost)
        # self.ampl.close()
    
    # Function to suppress output
    @contextlib.contextmanager
    def suppress_output(self):
        # Open devnull to suppress the output
        with open(os.devnull, 'w') as devnull:
            # Redirect stdout and stderr
            old_stdout = sys.stdout
            old_stderr = sys.stderr
            sys.stdout = devnull
            sys.stderr = devnull
            try:
                yield
            finally:
                # Restore original stdout and stderr
                sys.stdout = old_stdout
                sys.stderr = old_stderr
    
    def solve(self):
        with self.suppress_output():
            """Solve the optimization problem."""
            self.ampl.option["solver"] = "ipopt"
            self.ampl.set_option("ipopt_options", "outlev = 0 expect_infeasible_problem = yes bound_push = 0.001 bound_frac = 0.001 warm_start_init_point = yes")   #max_iter = 1000
            # self.ampl.set_option("ipopt_options", """outlev = 0 expect_infeasible_problem = yes bound_push = 0.01 bound_frac = 0.01 warm_start_init_point = yes mu_strategy = adaptive mu_oracle = loqo  """)   #max_iter = 1000 mu_strategy = adaptive mu_oracle = loqo max_soc = 4 halt_on_ampl_error = yes
            self.ampl.option["presolve_eps"] = "6.82e-14"
            self.ampl.option['presolve'] = 1
            # self.ampl.option['solver_msg'] = 0
            # self.ampl.option['show_stats'] = 0
            self.ampl.solve()
            self.solve_result = self.ampl.solve_result
            self.total_cost = self.ampl.get_objective("total_cost").value()
        # print("Objective:", self.total_cost)
        # print("solve_result: ",self.solve_result)
        solve_time = self.ampl.get_value('_solve_elapsed_time')
        self.solver_time += solve_time
        self.number_of_nlp += 1
        
    def run(self):
        """Main method to run the optimization process."""
        
        self.start_time = time.time()
        print("Solve the original nonconvex optimization problem using IPOPT ")
        self.load_model()
        self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
        # self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}1.2654*l[i,j,k]*(d[k])^1.327;")
        self.fix_leaf_arc_flow()
        
        self.super_source_out_arc = self.fix_arc_set()
        
        # self.ampl.eval("subject to con7{(i,j) in arcs}: -sum{k in nodes diff Source} D[k] <= q[i,j];")
        # self.ampl.eval("subject to con8{(i,j) in arcs}: q[i,j] <= sum{k in nodes diff Source} D[k];")
       
        # self.ampl.eval("display option ipopt_options;")     # Display all options in AMPL

        self.solve()
        
        print("Objective: ",self.total_cost)
        print("Solve_result: ",self.solve_result)
        print("Solve_time:", self.ampl.get_value('_solve_elapsed_time'),"\n")
        
        self.l = self.ampl.getVariable('l').getValues().to_dict()
        self.q = self.ampl.getVariable('q').getValues().to_dict()
        self.h = self.ampl.getVariable('h').getValues().to_dict()
        
        self.super_source_out_arc = self.fix_arc_set()
        self.network_graph = self.generate_random_acyclic_from_solution(self.q)
        
        # print("Fix the flow direction in optimization model and solve the updated model")
        
        self.inarc = []
        self.indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
        
        for node in self.indegree_2_or_more:
            for (u, v) in list(self.network_graph.in_edges(node)):
                if (u, v) in self.arcs:
                    self.inarc.append((u,v))
                else:
                    self.inarc.append((v,u))
        # print("inarc:", self.inarc)
        inarc_ = self.inarc
        
        inarc_set = []
        for (i, j) in self.inarc:
            if (i, j) in self.arcs:
                inarc_set.append(f"({i},{j})")
            else:
                inarc_set.append(f"({j},{i})")
        inarc_set = ", ".join(inarc_set)
        
        # self.display_results()
        self.iterate_acyclic_flows()
        elapsed_time = time.time() - self.start_time
        
        print("Solver_time:",self.solver_time, "seconds")
        print(f"Heuristic elapsed time:, {elapsed_time} seconds = {elapsed_time/60:.2f} minutes.")
        
if __name__ == "__main__":
    data_list = [
        "d1_Sample_input_cycle_twoloop",
        "d2_Sample_input_cycle_hanoi",
        "d3_Sample_input_double_hanoi",
        "d4_Sample_input_triple_hanoi",
        "d5_Taichung_input",
        "d6_HG_SP_1_4",
        "d7_HG_SP_2_3",
        "d8_HG_SP_3_4",
        "d9_HG_SP_4_2",
        "d10_HG_SP_5_5",
        "d11_HG_SP_6_3",
        "d12",
        "d13",
        "d14_NewYork",
        "d15_foss_poly_0",
        "d16_foss_iron",
        "d17_foss_poly_1",
        "d18_pescara",
        "d19_modena"
    ]
    
    # Select the data number here (0 to 18)
    data_number = 8
    input_data_file = f"/home/nitishdumoliya/waterNetwork/data/{data_list[data_number]}.dat"
    print("Water Network:", data_list[data_number],"\n")
    optimizer = WaterNetworkOptimizer("../m1Basic.mod", input_data_file, data_number, verbose=True)
    # optimizer = WaterNetworkOptimizer(sys.argv[1], sys.argv[3], sys.argv[2])
    optimizer.run()

Water Network: d9_HG_SP_4_2 

Solve the original nonconvex optimization problem using IPOPT 
Objective:  9032310.646773502
Solve_result:  solved
Solve_time: 10.21735 

sorted_nodes [138, 143, 134, 126, 135, 123, 17, 54, 112, 9, 83, 35, 59, 93, 105, 109, 85, 76, 95, 89, 43, 104, 2, 103, 114, 5, 60, 49, 42, 74, 111, 28, 58, 18, 68, 65, 57, 48, 23, 101, 6, 24, 36, 47, 53, 100, 78, 87, 63, 115]

******************************************************************************************************************************************
Iteration : 1 

sorted_delta_arc: [(33, 49), (25, 49), (126, 59), (37, 5), (114, 5), (32, 60), (67, 74), (51, 95), (43, 85), (69, 42), (22, 9), (80, 6), (29, 112), (44, 35), (29, 9), (91, 87), (105, 112), (24, 101), (75, 36), (8, 24), (113, 135), (36, 42), (53, 63), (141, 134), (3, 138), (117, 143), (108, 105), (55, 100), (77, 87), (31, 104), (115, 138), (11, 126), (60, 57), (26, 74), (60, 58), (5, 76), (76, 126), (119, 115), (59, 135), (93, 54), (95, 103), (87,

In [63]:
import networkx as nx
from amplpy import AMPL
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
import sys
import os
import contextlib
from sklearn.decomposition import PCA
import random

import matplotlib.lines as mlines
import matplotlib.patches as mpatches

import warnings
warnings.filterwarnings("ignore")

import multiprocessing
# !pip install pathos

# from pathos.multiprocessing import ProcessingPool as Pool

class WaterNetworkOptimizer:
    def __init__(self, model_file, data_file, data_number, verbose=True):
        self.ampl = AMPL()
        self.model_file = model_file
        self.data_file = data_file
        self.data_number = data_number
        self.total_cost = None
        self.network_graph = None
        self.solve_result = None
        self.solver_time = 0
        self.best_acyclic_flow = None
        self.number_of_nlp = 0
        self.verbose = verbose

    def load_model(self):
        """Load the model and data."""
        self.ampl.reset()
        self.ampl.read(self.model_file)
        self.ampl.read_data(self.data_file)
        
        self.nodes = self.ampl.getSet('nodes')
        self.source = self.ampl.getSet('Source')
        self.arcs = self.ampl.getSet('arcs')
        self.pipes = self.ampl.getSet('pipes')
        
        self.L = self.ampl.getParameter('L').to_dict()
        self.D = self.ampl.getParameter('D').to_dict()
        self.C = self.ampl.getParameter('C').to_dict()
        self.P = self.ampl.getParameter('P').to_dict()
        self.R = self.ampl.getParameter('R').to_dict()
        self.E = self.ampl.getParameter('E').to_dict()
        self.d = self.ampl.getParameter('d').to_dict()

        self.delta = 0.1
        self.p = 1.852
        self.omega = 10.67

    def create_digraph(self):
        nodes_list = [i for i in self.ampl.getSet('nodes')]
        edges_list = self.ampl.getSet('arcs').to_list()
        self.network_graph = nx.DiGraph()
        self.network_graph.add_nodes_from(nodes_list)
        self.network_graph.add_edges_from(edges_list)
        print(nodes_list)
        print(edges_list)

    def display_results(self):
        """Display relevant results from the optimization."""
        self.ampl.eval("display {(i,j) in arcs, k in pipes:l[i,j,k]>1} l[i,j,k];")
        self.ampl.eval("display {(i,j) in arcs}: q[i,j];")
        self.ampl.eval("display h;")
        self.ampl.eval("display solve_result;")
        self.total_cost = self.ampl.get_objective("total_cost").value()
        print("Objective:", self.total_cost)
        # self.ampl.eval("display {(i,j) in arcs} h[i] - h[j];")
        # self.ampl.eval("display {i in nodes} h[i] - (E[i] + P[i]);")

    # def plot_graph(self):
    #     print("Edges of the graph:",self.network_graph.edges())
    #     plt.figure(figsize=(10, 8))
    #     nx.draw_spectral(self.network_graph, with_labels=True)
    #     plt.show()


    def plot_graph1(self, super_source_out_arc=None, best_arc=None,current_cost = None, iteration= 1):
        # print("Edges of the graph:", self.network_graph.edges())
        indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
        
        plt.figure(figsize=(10, 8))
        pos = nx.spectral_layout(self.network_graph)
        nx.draw_networkx_nodes(self.network_graph, pos, node_color='lightblue', node_size=200)
        
        if indegree_2_or_more:
            nx.draw_networkx_nodes(self.network_graph, pos, nodelist=indegree_2_or_more, node_color='orange', node_size=200)
            
        nx.draw_networkx_labels(self.network_graph, pos)
        nx.draw_networkx_edges(self.network_graph, pos, edge_color='black')
        
        if super_source_out_arc:
            nx.draw_networkx_edges(self.network_graph, pos, edgelist=super_source_out_arc, edge_color='red', width=1)
            
        if best_arc:
            nx.draw_networkx_edges(self.network_graph, pos, edgelist=[best_arc], edge_color='magenta', width=1)
            
        plt.title(f"total_cost:{self.total_cost}")
        plt.savefig(f"/home/nitishdumoliya/waterNetwork/model/figure/d{self.data_number}_iteration_{iteration}.png")
        plt.box(False)
        plt.show()

    def plot_graph(self, super_source_out_arc=None, current_cost = None, iteration = 1, edge_weights= None, h = None, D = None, arc=(0,0), l ={}, C = {}):
        # self.network_graph = nx.DiGraph()
        # self.network_graph.add_edges_from(edges)
        # Node positions as per your specifications
        
        new_positions0 = {
             2 : (2000.00, 3000.00),
             3 : (1000.00, 3000.00),        
             4 : (2000.00, 2000.00),        
             5 : (1000.00, 2000.00),       
             6 : (2000.00, 1000.00),       
             7 : (1000.00, 1000.00),      
             1 : (3000.00, 3000.00)     
            }  
        
        new_positions1 = {
            2: (6000.00, 2000.00),
            3: (6000.00, 4000.00),
            4: (7500.00, 4000.00),
            5: (9000.00, 4000.00),
            6: (10000.00, 4000.00),
            7: (10000.00, 6000.00),
            8: (10000.00, 8000.00),
            9: (10000.00, 10000.00),
            10: (9000.00, 10000.00),
            11: (9000.00, 11500.00),
            12: (9000.00, 13000.00),
            13: (7000.00, 13000.00),
            14: (8000.00, 10000.00),
            15: (7000.00, 10000.00),
            16: (6000.00, 10000.00),
            17: (6000.00, 8500.00),
            18: (6000.00, 7000.00),
            19: (6000.00, 5500.00),
            20: (4500.00, 4000.00),
            21: (4500.00, 2000.00),
            22: (4500.00, 0.00),
            23: (3000.00, 4000.00),
            24: (3000.00, 7000.00),
            25: (3000.00, 10000.00),
            26: (4000.00, 10000.00),
            27: (5000.00, 10000.00),
            28: (1500.00, 4000.00),
            29: (0.00, 4000.00),
            30: (0.00, 7000.00),
            31: (0.00, 10000.00),
            32: (1500.00, 10000.00),
            1: (8000.00, 0.00)
        }       
        
        new_positions16 = {
            2: (5679.61, 9538.83),
            3: (4862.46, 9538.83),
            4: (2750.81, 9474.11),
            5: (1852.75, 8357.61),
            6: (1974.11, 6076.05),
            7: (1974.11, 5149.68),
            8: (4235.44, 5076.86),
            9: (6411.81, 5093.04),
            10: (5412.62, 7888.35),
            11: (4510.52, 8264.56),
            12: (3033.98, 9243.53),
            13: (2301.78, 8078.48),
            14: (2944.98, 7669.90),
            15: (3786.41, 7139.97),
            16: (4830.10, 6480.58),
            17: (7099.51, 8438.51),
            18: (5505.66, 8450.65),
            19: (3563.92, 8839.00),
            20: (3167.48, 7532.36),
            21: (2730.58, 7285.60),
            22: (3511.33, 6666.67),
            23: (4097.90, 6286.41),
            24: (3337.38, 5121.36),
            25: (4530.74, 6011.33),
            26: (4215.21, 7783.17),
            27: (5194.17, 7055.02),
            28: (5218.45, 5089.00),
            29: (5622.98, 5999.19),
            30: (5950.65, 5796.93),
            31: (6614.08, 7621.36),
            32: (5380.26, 7544.50),
            33: (6318.77, 7281.55),
            34: (6549.35, 7212.78),
            35: (6585.76, 6092.23),
            36: (7152.10, 6104.37),
            1: (7111.65, 7532.36),
            37: (7669.90, 7783.17)
        }

        # # Update node positions
        pos = new_positions0
        # pos = nx.spectral_layout(self.network_graph)

        cost = {}

        for (i,j) in self.ampl.getSet('arcs'):
            cost[i,j] = sum(l[i,j,k] * C[k] for k in self.ampl.getSet('pipes'))
        
        plt.figure(figsize=(10, 8))
        cmap = plt.cm.plasma

        nx.draw_networkx_nodes(self.network_graph, pos, node_color='lightblue', node_size=300, label='Regular Nodes')

        indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
        if indegree_2_or_more:
            nx.draw_networkx_nodes(self.network_graph, pos, nodelist=indegree_2_or_more, node_color='orange', node_size=300, label='Nodes with In-Degree ≥ 2')

        nx.draw_networkx_labels(self.network_graph, pos, font_size=10)

        nx.draw_networkx_edges(self.network_graph, pos, arrowstyle="->", arrowsize=12, edge_color='gray', label='Regular Arcs')

        if super_source_out_arc:
            nx.draw_networkx_edges(self.network_graph, pos, edgelist=super_source_out_arc,arrowstyle="->", arrowsize=12, edge_color='red', width=1, label='Fix arc direction')

        # if best_arc:
        #     nx.draw_networkx_edges(self.network_graph, pos, edgelist=[best_arc],arrowstyle="->", arrowsize=12, edge_color='magenta', width=1, label = 'Best arc')
        # Annotate node demands
        if h:
            for node, (x, y) in pos.items():
                demand = h.get(node, 0)  # Get the head for the node, default to 0 if not in dictionary
                plt.text(x, y + 250, f"{demand:.2f}", fontsize=8, color='blue', ha='center')  # Annotate demand below the node
                # plt.text(x, y + 100, f"{demand:.2f}", fontsize=8, color='blue', ha='center')  # Annotate demand below the node
                # plt.text(x, y -100 , f"{demand:.2f}", fontsize=8, color='magenta', ha='center')  # Annotate demand below the node
                # plt.text(mid_x, mid_y + 50 , f"{weight:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
                # plt.text(mid_x, mid_y + 100, f"{value:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        if D:
            for node, (x, y) in pos.items():
                demand = D.get(node, 0)  # Get the demand for the node, default to 0 if not in dictionary
                plt.text(x, y - 300, f"{demand:.2f}", fontsize=8, color='magenta', ha='center')  # Annotate demand below the node
                # plt.text(x, y -100 , f"{demand:.2f}", fontsize=8, color='magenta', ha='center')  # Annotate demand below the node
            

        if edge_weights:
            for (u, v), weight in edge_weights.items():
                # if self.network_graph.has_edge(u, v):
                mid_x = (pos[u][0] + pos[v][0]) / 2  # Midpoint x-coordinate
                mid_y = (pos[u][1] + pos[v][1]) / 2  # Midpoint y-coordinate
                # plt.text(mid_x, mid_y+100 , f"{weight:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
                plt.text(mid_x, mid_y + 50 , f"{weight:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        if cost:
            for (u, v), value in cost.items():
                # if self.network_graph.has_edge(u, v):
                mid_x = (pos[u][0] + pos[v][0]) / 2  # Midpoint x-coordinate
                mid_y = (pos[u][1] + pos[v][1]) / 2  # Midpoint y-coordinate
                # plt.text(mid_x, mid_y-300 , f"{round(value)}", fontsize=8, color='purple', ha='center')  # Annotate weight on edge
                # plt.text(mid_x, mid_y + 100, f"{value:.2f}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        pipe_dia_arc = {}
        for (i,j) in self.arcs:
            list_=[]
            for k in self.pipes:
                if l[i,j,k]>= 1e-5:
                    list_.append(k)
            pipe_dia_arc[i,j] = list_
        
        if pipe_dia_arc:
            for (u, v), weight in pipe_dia_arc.items():
                # if self.network_graph.has_edge(u, v):
                mid_x = (pos[u][0] + pos[v][0]) / 2  # Midpoint x-coordinate
                mid_y = (pos[u][1] + pos[v][1]) / 2  # Midpoint y-coordinate
                # plt.text(mid_x, mid_y+550 , f"{weight}", fontsize=8, color='purple', ha='center')  # Annotate weight on edge
                plt.text(mid_x, mid_y+150 , f"{weight}", fontsize=8, color='green', ha='center')  # Annotate weight on edge
        
        regular_node_patch = mpatches.Patch(color='lightblue', label='Regular Nodes')
        indegree_node_patch = mpatches.Patch(color='orange', label='Nodes with In-Degree ≥ 2')
        regular_edge_line = mlines.Line2D([], [], color='black', label='Regular Arcs')
        super_source_edge_line = mlines.Line2D([], [], color='red', label='Fix arc direction')
        # best_edge_line = mlines.Line2D([], [], color='magenta', label='Best Arc')
        plt.legend(handles=[regular_node_patch, indegree_node_patch, regular_edge_line, super_source_edge_line], loc='lower right')
        
        cost = round(self.total_cost)
        # res = f"{cost:,}"
        plt.title(f"total_cost:{self.format_indian_number(cost)}")
        # (u,v) = arc
        # plt.savefig(f"/home/nitishdumoliya/waterNetwork/model/figure/d{self.data_number}_iteration_{iteration}.png")
        plt.box(True)
        plt.show()

    
    def cycle_basis(self):
        root = self.ampl.getSet('Source').to_list()[0]
        nodes_list = [i for i in self.ampl.getSet('nodes')]
        edges_list = self.ampl.getSet('arcs').to_list() 
        uwg = nx.Graph()
        uwg.add_nodes_from(nodes_list)
        uwg.add_edges_from(edges_list)
        # print("Edges in the undirected graph:", edges_list)
        print("cycle basis for given water network: ",nx.cycle_basis(uwg, root))
        
    def generate_random_acyclic_from_solution(self, q):
        # print("Generate the acyclic network using ipopt solution")
        
        self.network_graph = nx.DiGraph()
        self.network_graph.add_nodes_from(self.nodes)
        
        # q = self.ampl.getVariable('q').getValues().toDict()
        for (i,j) in self.arcs:
            if q[i,j] >= 0:
                self.network_graph.add_edge(i,j)
            else:
                self.network_graph.add_edge(j,i)
        
        return self.network_graph

    
    def generate_random_acyclic_graph(self):
        uwg = nx.Graph()
        nodes_list = [i for i in self.ampl.getSet('nodes')]
        edges_list = self.ampl.getSet('arcs').to_list()
        uwg.add_nodes_from(nodes_list)
        uwg.add_edges_from(edges_list)
        print("Edges in the undirected graph:", edges_list)
        
        # Generate a random spanning tree using Wilson's algorithm
        random_tree = nx.random_spanning_tree(uwg)
        
        # Retrieve the root from the AMPL source set
        root_l = self.ampl.getSet('Source').to_list()
        root = root_l[0]
        print("Root node:", root)

        # Ensure the root is present in the random tree
        if root not in random_tree.nodes:
            raise ValueError("The specified root must be a node in the graph.")

        # Create a directed graph from the random tree starting from the specified root
        self.network_graph = nx.DiGraph()
        visited = set()

        def dfs(node):
            visited.add(node)
            for neighbor in random_tree.neighbors(node):
                if neighbor not in visited:
                    self.network_graph.add_edge(node, neighbor) 
                    dfs(neighbor)

        # Start DFS from the specified root
        dfs(root)

        # Draw the initial directed tree
        plt.figure(figsize=(15, 10))
        plt.subplot(121)
        nx.draw_spectral(self.network_graph, with_labels=True, node_color='lightgreen', font_weight='bold', arrows=True)
        plt.title("Directed Spanning Tree")

        # Add remaining edges from the original graph and check for cycles
        for u, v in uwg.edges():
            if not self.network_graph.has_edge(u, v):  
                self.network_graph.add_edge(u, v)  
                if not nx.is_directed_acyclic_graph(self.network_graph):  
                    self.network_graph.remove_edge(u, v)  
                    self.network_graph.add_edge(v, u)  

        # Draw the final directed graph after adding remaining edges
        plt.subplot(122)
        nx.draw_spectral(self.network_graph, with_labels=True, node_color='lightgreen', font_weight='bold', arrows=True)
        plt.title("Acyclic Directed Graph")
        plt.show()

    def update_model(self):
        # print("Fix the arcs direction using the acyclic network\n")
        edges_list = [(arc[0],arc[1]) for arc in self.ampl.getSet('arcs')]
        for edge in self.network_graph.edges:
            i, j = edge
            if edge in edges_list:
                self.ampl.eval(f"s.t. flow_direction{i}_{j}: q[{i},{j}] >=0;")
                # self.ampl.eval(f"s.t. head_bound_left{i}_{j}: E[{j}]+P[{j}] <= h[{j}];")
                # self.ampl.eval(f"s.t. head_bound_right{i}_{j}: E[{j}] + P[{j}] <= h[{i}];")
            else:
                self.ampl.eval(f"s.t. flow_direction{i}_{j}: q[{j},{i}] <=0;")
                # self.ampl.eval(f"s.t. head_bound_left{i}_{j}: E[{i}]+P[{i}] <= h[{i}];")
                # self.ampl.eval(f"s.t. head_bound_right{i}_{j}: E[{i}] + P[{i}] <= h[{j}];")  


    def is_valid_edge(self, source, target):
        """Check if adding the directed edge (source -> target) maintains acyclicity."""
        self.network_graph.add_edge(source, target)  # Temporarily add the edge
        is_dag = nx.is_directed_acyclic_graph(self.network_graph)  # Check for acyclicity
        self.network_graph.remove_edge(source, target)  # Remove the edge after checking
        return is_dag  # Return True if it maintains acyclicity     
    
    def check_incoming_arcs(self):
        root = list(self.ampl.getSet('Source'))[0]
        # Iterate over all nodes in the graph
        for node in self.network_graph.nodes():
            # Skip the root node
            if node == root:
                continue
            # Check if the in-degree of the node is at least 1
            if self.network_graph.in_degree(node) < 1:
                # print(f"Node {node} does not have any incoming arcs.")
                return False
        # print("All nodes except the root have at least one incoming arc.")
        return True
            
    def format_indian_number(self,num):
        num_str = str(num)
        if len(num_str) <= 3:
            return num_str
        else:
            # Split the number into the last three digits and the rest
            last_three = num_str[-3:]
            remaining = num_str[:-3]
            # Add commas every two digits in the remaining part
            remaining = ','.join([remaining[max(i - 2, 0):i] for i in range(len(remaining), 0, -2)][::-1])
            return remaining + ',' + last_three

    def fix_leaf_arc_flow(self):
        graph = nx.Graph()
        arc_set = self.ampl.getSet('arcs').to_list()  
        graph.add_edges_from(arc_set)
        D = self.ampl.getParameter('D').getValues().to_dict()  
        source = self.ampl.getSet('Source').to_list()[0]
        fixed_arcs = set()
        # print("\nPresolve the model for fixing the flow value in the leaf arcs")
        # print("Source:",self.ampl.getSet('Source').to_list())

        while True:
            leaf_nodes = [node for node in graph.nodes if graph.degree[node] == 1]
            # print("leaf_nodes:", leaf_nodes)
            if not leaf_nodes:  
                break

            for leaf in leaf_nodes:
                neighbor = next(graph.neighbors(leaf))
                if (neighbor, leaf) in arc_set:
                    edge = (neighbor, leaf)
                    if edge not in fixed_arcs:  
                        if leaf == source:
                            flow_value = D[leaf]
                            D[neighbor] = (D[neighbor]+flow_value)
                            source = neighbor
                        else:
                            flow_value = D[leaf]
                            D[neighbor] = D[neighbor] + flow_value
                        self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        # print(f"Fixing flow for arc {edge}: {flow_value}")
                        fixed_arcs.add(edge)  

                    graph.remove_node(leaf)
                else:
                    edge = (leaf, neighbor)
                    if edge not in fixed_arcs:  
                        if leaf == source:
                            flow_value = -D[leaf]
                            D[neighbor] = D[neighbor]-flow_value
                            source = neighbor
                            self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        elif neighbor == source:
                            flow_value = -D[leaf]
                            D[neighbor] = D[neighbor] - D[leaf] 
                            self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        else:
                            flow_value = -D[leaf]
                            D[neighbor] += -flow_value
                            self.ampl.eval(f"s.t. fix_q_{edge[0]}_{edge[1]}: q[{edge[0]},{edge[1]}] = {flow_value};")
                        # print(f"Fixing flow for arc {edge}: {flow_value}")
                        fixed_arcs.add(edge)  
                    graph.remove_node(leaf)
        # print("All leaf arc flows have been fixed.")


    def is_cycle(self, graph, start_node, end_node, visited_copy, parent):
        visited_copy[start_node] = True
        # print(f"Is node {end_node} in cycle?")
        for neighbor in graph.neighbors(start_node):
            # print("visited",neighbor,visited_copy[neighbor])
            if not visited_copy[neighbor]:
                # print("neighbor of node", start_node, "is", neighbor)
                isCycle = self.is_cycle(graph, neighbor, end_node, visited_copy, start_node)
                if isCycle:
                    return True
            else:
                # print("parent:", parent)
                if parent != neighbor:
                    if end_node == neighbor:
                        # print(f"Node {end_node} is in cycle")
                        return True
        return False

    def presolve(self, graph, node, visited, parent, set_arc):
        visited_copy = visited.copy()
        # print(visited_copy)
        isCycle = self.is_cycle(graph, node, node, visited_copy, parent)
        # print(f"Is node {node} in cycle?",isCycle)
        visited[node] = True
        if isCycle:
            for neighbor in graph.neighbors(node):
                if parent!=neighbor:
                    set_arc.append((node,neighbor))
                    # print("Fix the arc", (node, neighbor))
            return set_arc
        else:
            for neighbor in graph.neighbors(node):
                if parent != neighbor:
                    set_arc.append((node,neighbor))
                    # print(set_arc)
                    # print("Fix the arc", (node, neighbor))
                    # print("neighbor:", neighbor)
                    self.presolve(graph, neighbor, visited, node, set_arc)
        return set_arc

    def fix_arc_set(self):
        graph = nx.Graph()
        arc_set = self.ampl.getSet('arcs').to_list()
        graph.add_edges_from(arc_set)
        visited = {node: False for node in graph.nodes()}
        source = self.ampl.getSet('Source').to_list()[0]
        set_arc = []
        # print("\nPresolve the model for fixing the arc direction")
        set_ = self.presolve(graph, source, visited, -1, set_arc)
        # print("fixed arc direction:",set_, "\n") 
        return set_


    def update_initial_points(self,l_solution, q_solution, h_solution):
        for (i, j, k), val in l_solution.items():
            self.ampl.eval(f'let l[{i},{j},{k}] := {val};')
        for (i, j), val in q_solution.items():
            self.ampl.eval(f'let q[{i},{j}] := {val};')
        for i, val in h_solution.items():
            self.ampl.eval(f'let h[{i}] := {val};')

    def update_initial_points1(self, l_solution, q_solution, h_solution, t_solution,all_duals, inarc):
        for (i, j, k), val in l_solution.items():
            self.ampl.eval(f'let l[{i},{j},{k}] := {val};')
        
        edge_list_network = self.network_graph.edges
        
        for i, val in h_solution.items():
            self.ampl.eval(f'let h[{i}] := {val};')
        
        for (i, j), val in q_solution.items():
            edge = (i, j) 
            if edge in edge_list_network:
                if (i,j) not in inarc:
                    # print(f"self.ampl.eval(let q[{i},{j}] := {val};)")
                    self.ampl.eval(f"let q[{i},{j}] := {val} ;")
                else:
                    # print(f"self.ampl.eval(let q[{i},{j}] := {-val};)")
                    self.ampl.eval(f"let q[{i},{j}] := {val} ;")
            else:
                if (j,i) not in inarc:
                    # print(f"self.ampl.eval(let q[{i},{j}] := {val};)")
                    self.ampl.eval(f"let q[{i},{j}] := {val};")
                else:
                    # print(f"self.ampl.eval(let q[{j},{i}] := {-val};)")
                    self.ampl.eval(f"let q[{j},{i}] := {val} ;")
        for i, val in t_solution.items():
            self.ampl.eval(f'let t[{i}] := {val};')
            
        current_duals = {}
        for con_name, val in self.ampl.get_constraints():
            dual_values = val.get_values()
            current_duals[con_name] = dual_values

        # Initialize dual values for all constraints
        # for con_name, dual_values in all_duals.items():
            # if con_name in current_duals:
                # Initialize dual values for each constraint
                # self.ampl.get_constraint(con_name).set_values(dual_values)
            # else:
            #     print(f"Skipping initialization for constraint: {con_name} (not in current model)")
    
    def update_initial_points_with_perturbation(self, l_solution, q_solution, h_solution,all_duals, inarc, delta=0.1):
        edge_list_network = self.network_graph.edges
        L = self.ampl.getParameter('L').getValues().to_dict()
        # Perturb l values
        for (i, j, k), val in l_solution.items():
            if (i,j) not in inarc:
                if val>= 1e-5:
                    perturbation = random.gauss(0, 1)
                    new_val = val + perturbation
                    # if val >= 1e-5:
                    self.ampl.eval(f'let l[{i},{j},{k}] := {new_val};')
                else:
                    self.ampl.eval(f'let l[{i},{j},{k}] := {0};')
            else:
                if val>= 1e-5:
                    perturbation = random.gauss(0, 1)
                    new_val = val + perturbation
                    self.ampl.eval(f'let l[{i},{j},{k}] := {new_val};')
                else:
                    self.ampl.eval(f'let l[{i},{j},{k}] := {0};')
                    
        # Perturb h values
        for i, val in h_solution.items():
            perturbation = random.gauss(0, 1)
            new_val = val + perturbation
            self.ampl.eval(f'let h[{i}] := {new_val};')

        # Modify q values based on heuristic
        for (i, j), val in q_solution.items():
            edge = (i, j)
            perturbation = random.gauss(0, 1)
            if edge in edge_list_network:
                if (i, j) not in inarc:
                    self.ampl.eval(f"let q[{i},{j}] := {val + perturbation};")
                else:
                    self.ampl.eval(f"let q[{i},{j}] := {(val + perturbation)};")
            else:
                if (j, i) not in inarc:
                    self.ampl.eval(f"let q[{i},{j}] := {val + perturbation};")
                else:
                    self.ampl.eval(f"let q[{j},{i}] := {(val + perturbation)};")
        
        current_duals = {}
        for con_name, val in self.ampl.get_constraints():
            dual_values = val.get_values()
            current_duals[con_name] = dual_values

        # Initialize dual values for all constraints
        for con_name, dual_values in all_duals.items():
            if con_name in current_duals:
                # Initialize dual values for each constraint
                self.ampl.get_constraint(con_name).set_values(dual_values)

    def acyclic_arcs(self):
        network_graph = self.best_acyclic_flow
        indegree_2_or_more = [node for node, indeg in network_graph.in_degree() if indeg >= 2]
        acyclic_arc = set()
        for node in indegree_2_or_more:
            # print("Node:", node,"in_degree:", self.network_graph.in_degree(node))
            for edge in list(network_graph.in_edges(node)):
                (u, v) = edge
                if (u,v) not in self.super_source_out_arc :
                    network_graph.remove_edge(u,v)
                    network_graph.add_edge(v,u)
                    acy_check = nx.is_directed_acyclic_graph(network_graph)
                    in_arc_check = self.check_incoming_arcs()
                    # print("Acyclic", acy_check and in_arc_check)
                    if acy_check and in_arc_check:
                        acyclic_arc.add((u,v))

                    network_graph.remove_edge(v, u)
                    network_graph.add_edge(u, v) 
        return acyclic_arc


    def process_arc(self, arc):
        u, v = arc
        # Reverse the arc
        self.network_graph.remove_edge(u, v)
        self.network_graph.add_edge(v, u)
        acy_check = nx.is_directed_acyclic_graph(self.network_graph)
        in_arc_check = self.check_incoming_arcs()

        if acy_check and in_arc_check:
            self.load_model()
            self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
            self.fix_leaf_arc_flow()
            self.update_initial_points1(self.l, self.q, self.h, self.t, self.all_duals, self.inarc)

            if (u, v) in self.arcs:
                self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{u}, {v}]<=0;")
            else:
                self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{v}, {u}]>=0;")

            self.solve()

            if self.solve_result == "solved":
                l = self.ampl.getVariable('l').getValues().to_dict()
                q = self.ampl.getVariable('q').getValues().to_dict()
                h = self.ampl.getVariable('h').getValues().to_dict()
                t = self.ampl.getVariable('t').getValues().to_dict()

                return {
                    "arc": arc,
                    "acyclic": True,
                    "total_cost": self.total_cost,
                    "solve_result": self.solve_result,
                    "solve_time": round(self.ampl.get_value('_solve_elapsed_time'), 2),
                    "improved": self.total_cost < self.current_cost,
                    "l": l,
                    "q": q,
                    "h": h,
                    "t": t,
                    "acyclic_flow": self.network_graph.copy()
                }

        # Revert the arc if not successful
        self.network_graph.remove_edge(v, u)
        self.network_graph.add_edge(u, v)
        return {"arc": arc, "acyclic": False}

    def run_parallel1(self):
        improved = False
        best_result = None
        
        while True:
            # Use multiprocessing to process arcs in parallel
            with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
                results = pool.map(self.process_arc, self.acyclic_arc_set)

            # Filter valid solutions and find the best result
            valid_results = [r for r in results if r["acyclic"] and "total_cost" in r]
            if valid_results:
                best_result = min(valid_results, key=lambda r: r["total_cost"])

                if best_result["total_cost"] < self.current_cost:
                    self.current_cost = best_result["total_cost"]
                    self.best_acyclic_flow = best_result["acyclic_flow"]
                    self.l = best_result["l"]
                    self.q = best_result["q"]
                    self.h = best_result["h"]
                    self.t = best_result["t"]
                    improved = True
                    
                    print(f"New best solution found! Total cost: {self.current_cost}")
                else:
                    improved = False
            else:
                print("No improved solutions found in this iteration.")
            
            # If no improvement, stop the loop
            if not improved:
                break

            # Update the graph and prepare for the next iteration
            self.network_graph = self.best_acyclic_flow.copy()
            self.acyclic_arc_set = self.acyclic_arcs()  # Recompute the arcs to process
        
        # Return the best result
        return best_result, improved
    
    def iterate_arc(self, iteration, improved, current_cost, best_arc):
        improved = False
        self.network_graph = self.best_acyclic_flow.copy()
        
        # print("Acyclic network arcs direction: ",self.network_graph.edges())
        # print("Fixed arc set:",self.super_source_out_arc)
        
        BEST_ARC = []
        # BEST_ARC.append(best_arc)
        
        self.inarc = []
        for node in self.indegree_2_or_more:
            for (u, v) in list(self.network_graph.in_edges(node)):
                if (u, v) in self.arcs:
                    self.inarc.append((u,v))
                else:
                    self.inarc.append((v,u))
        print("inarc:", self.inarc)
        inarc_ = self.inarc
        
        inarc_set = []
        for (i, j) in self.inarc:
            if (i, j) in self.arcs:
                inarc_set.append(f"({i},{j})")
            else:
                inarc_set.append(f"({j},{i})")
        
        # Convert the list into a string compatible with AMPL
        inarc_set = ", ".join(inarc_set)
        
        # for node in self.indegree_2_or_more:
        #     print("Node:", node,"in_degree:", self.network_graph.in_degree(node))
        #     for u,v in list(self.network_graph.in_edges(node)):
        for (u,v) in self.acyclic_arc_set:
            self.network_graph.remove_edge(u,v)
            self.network_graph.add_edge(v,u)
            acy_check = nx.is_directed_acyclic_graph(self.network_graph)
            in_arc_check = self.check_incoming_arcs()
            # print("Acyclic", acy_check and in_arc_check)
            if acy_check and in_arc_check:
                #l_sol, q_sol, h_sol = self.generate_initial_points()
                self.load_model()
                # self.ampl.eval(f"set inarc := {{{inarc_set}}};")
                # self.ampl.eval(f"set indegree_node := {{{set(self.indegree_2_or_more)}}};")
                self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
                self.fix_leaf_arc_flow()
                self.update_initial_points1(self.l, self.q, self.h, self.t, self.all_duals, self.inarc)
                # self.update_model()
                self.ampl.eval(f"param Q_max = sum{{k in nodes diff Source }} D[k];")   
                # self.ampl.eval("subject to con7{(i,j) in arcs}: -sum{k in nodes diff Source} D[k] <= q[i,j];")
                # self.ampl.eval("subject to con8{(i,j) in arcs}: q[i,j] <= sum{k in nodes diff Source} D[k];")
                
                if (u,v) in self.arcs:
                    self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{u}, {v}]<=0;")
                    self.ampl.eval(f"s.t. flow_bound_left_{u}_{v}: -Q_max <= q[{u}, {v}];")
                else:
                    self.ampl.eval(f"s.t. flow_direction{u}_{v}: q[{v}, {u}]>=0;")
                    self.ampl.eval(f"s.t. flow_bound_right_{u}_{v}: q[{v}, {u}] <= Q_max;")
                self.solve()
                
                l = self.ampl.getVariable('l').getValues().to_dict()
                q = self.ampl.getVariable('q').getValues().to_dict()
                h = self.ampl.getVariable('h').getValues().to_dict()
                t = self.ampl.getVariable('t').getValues().to_dict()
                
                if self.solve_result == "solved":
                    
                    if self.total_cost < current_cost:
                        # print("Arc", (u,v),"Acyclic:", acy_check and in_arc_check, "Best optimal: ", '{:,}'.format(round(current_cost)), "New optimal: ", '{:,}'.format(round(self.total_cost)), "Solve_time:", self.ampl.get_value('_solve_elapsed_time'), "Solve_result: ", self.solve_result, "Improved: Yes")
                        
                        print(f"Arc: {(u,v)}",
                              f"Acyclic: {acy_check and in_arc_check}",
                              f"Best_optimal: {self.format_indian_number(round(current_cost))}", 
                              f"New_optimal: {self.format_indian_number(round(self.total_cost))}", 
                              f"Solve_time: {round(self.ampl.get_value('_solve_elapsed_time'), 2)}", 
                              f"Solve_result: {self.solve_result}", "Improved: Yes", 
                              f"Time_count: {round(time.time() - self.start_time, 2)}")

                        current_cost = self.total_cost
                        improved = True
                        self.network_graph = self.generate_random_acyclic_from_solution(q)
                        
                        self.best_acyclic_flow = self.network_graph.copy()
                        self.indegree_2_or_more = [node for node, indeg in self.best_acyclic_flow.in_degree() if indeg >= 2]
                        # print("indegree_2_or_more:", self.indegree_2_or_more)
                        
                        best_arc = (v,u)
                        self.l = l 
                        self.q = q
                        self.h = h 
                        self.t = t

                        self.all_duals = {}
                        for con_name, val in self.ampl.get_constraints():
                            # Get dual values for each constraint
                            dual_values = val.getValues()
                            self.all_duals[con_name] = dual_values

                        self.inarc = []
                        for node in self.indegree_2_or_more:
                            for (u, v) in self.best_acyclic_flow.in_edges(node):
                                if (u, v) in self.arcs:
                                    if (u,v) not in self.super_source_out_arc:
                                        self.inarc.append((u,v))
                                else:
                                    self.inarc.append((v,u))
                        # self.plot_graph(self.super_source_out_arc, current_cost, iteration, self.q, self.h, self.D, self.l, self.C)
                        self.acyclic_arc_set = self.acyclic_arcs()
                        
                        break
                    else:
                        # print("Arc", (u,v),"Acyclic:", acy_check and in_arc_check, "Best optimal: ", '{:,}'.format(round(current_cost)), "New optimal: ", '{:,}'.format(round(self.total_cost)), "Solve_time:", self.ampl.get_value('_solve_elapsed_time'), "Solve_result: ", self.solve_result, "Improved: No")

                        print(f"Arc: {(u,v)}",
                              f"Acyclic: {acy_check and in_arc_check}",
                              f"Best_optimal: {self.format_indian_number(round(current_cost))}", 
                              f"New_optimal: {self.format_indian_number(round(self.total_cost))}", 
                              f"Solve_time: {round(self.ampl.get_value('_solve_elapsed_time'), 2)}", 
                              f"Solve_result: {self.solve_result}", "Improved: No ", 
                              f"Time_count: {round(time.time() - self.start_time, 2)}")
                        
                        self.network_graph.remove_edge(v, u)
                        self.network_graph.add_edge(u, v)  
                else:
                    # print("Arc", (u,v),"Acyclic:",acy_check and in_arc_check, "Best optimal: ", '{:,}'.format(round(current_cost)), "New optimal: ", '{:,}'.format(round(self.total_cost)),  "Solve_time:", self.ampl.get_value('_solve_elapsed_time'), "Solve_result: ", self.solve_result)
                    print(f"Arc: {(u,v)}",
                          f"Acyclic: {acy_check and in_arc_check}",
                          f"Best_optimal: {self.format_indian_number(round(current_cost))}", 
                          f"New_optimal: {self.format_indian_number(round(self.total_cost))}", 
                          f"Solve_time: {round(self.ampl.get_value('_solve_elapsed_time'), 2)}", 
                          f"Solve_result: {self.solve_result}", "Improved: No ", 
                          f"Time_count: {round(time.time() - self.start_time, 2)}")
                    self.network_graph.remove_edge(v, u)
                    self.network_graph.add_edge(u, v)                         
            else:
                print(f"Arc: {(u,v)}",
                      f"Acyclic: {acy_check and in_arc_check}")
                
                self.network_graph.remove_edge(v, u)
                self.network_graph.add_edge(u, v)                      
            print(" ")
            if improved:
                break
        return self.best_acyclic_flow, improved, current_cost, self.l, self.q, self.h, best_arc

    
    @staticmethod
    def worker(data, stop_flag):
        # (arc, network_graph) = data
        # verbose=False
        # with contextlib.redirect_stdout(None):
        (arc, network_graph,current_cost, best_acyclic_flow, data_number, l, q, h) = data
        
        u, v = arc
        
        # print(arc)
        # print(ampl_config)
        data_list = [
            "d1_Sample_input_cycle_twoloop",
            "d2_Sample_input_cycle_hanoi",
            "d3_Sample_input_double_hanoi",
            "d4_Sample_input_triple_hanoi",
            "d5_Taichung_input",
            "d6_HG_SP_1_4",
            "d7_HG_SP_2_3",
            "d8_HG_SP_3_4",
            "d9_HG_SP_4_2",
            "d10_HG_SP_5_5",
            "d11_HG_SP_6_3",
            "d12",
            "d13",
            "d14_NewYork",
            "d15_foss_poly_0",
            "d16_foss_iron",
            "d17_foss_poly_1",
            "d18_pescara",
            "d19_modena"
        ]
        # Reverse the arc in a copy of the graph
        network_graph.remove_edge(u, v)
        network_graph.add_edge(v, u)
        
        # Create a new AMPL instance and initialize
        ampl = AMPL()
        ampl.read("../m1Basic.mod")
        ampl.read_data(f"/home/nitishdumoliya/waterNetwork/data/{data_list[data_number]}.dat")
        
        nodes = ampl.getSet('nodes')
        source = ampl.getSet('Source')
        arcs = ampl.getSet('arcs')
        pipes = ampl.getSet('pipes')
        
        # print(nodes)
        # print(arcs)
        # print(ampl_config["current_cost"])
        
        # L = ampl.getParameter('L').to_dict()
        # D = ampl.getParameter('D').to_dict()
        # C = ampl.getParameter('C').to_dict()
        # P = ampl.getParameter('P').to_dict()
        # R = ampl.getParameter('R').to_dict()
        # E = ampl.getParameter('E').to_dict()
        # d = ampl.getParameter('d').to_dict()

        delta = 0.1
        p = 1.852
        omega = 10.67
        
        ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
        # ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}1.2654*l[i,j,k]*(d[k])^1.327;")
        # self.fix_leaf_arc_flow()
        # self.update_initial_points1(ampl_config["l"], ampl_config["q"], ampl_config["h"], ampl_config["t"], ampl_config["all_duals"], ampl_config["inarc"])
        
        for (i, j, k), val in l.items():
            ampl.eval(f'let l[{i},{j},{k}] := {val};')
        for (i, j), val in q.items():
            ampl.eval(f'let q[{i},{j}] := {val};')
        for i, val in h.items():
            ampl.eval(f'let h[{i}] := {val};')
            
        # Add constraints for the arc direction
        if (u, v) in arcs:
            ampl.eval(f"s.t. flow_direction{u}_{v}: q[{u}, {v}]<=0;")
        else:
            ampl.eval(f"s.t. flow_direction{u}_{v}: q[{v}, {u}]>=0;")
            
        # ampl.eval("subject to con7{(i,j) in arcs}: -sum{k in nodes diff Source} D[k] <= q[i,j];")
        # ampl.eval("subject to con8{(i,j) in arcs}: q[i,j] <= sum{k in nodes diff Source} D[k];")
       
        ampl.option["solver"] = "ipopt"
        ampl.option["snopt_options"] = "major_iterations_limit=1000"
        # ampl.set_option("ipopt_options", "outlev = 0 expect_infeasible_problem = yes bound_push = 0.01 bound_frac = 0.01 warm_start_init_point = yes max_iter = 400")   #max_iter = 1000
        ampl.set_option("ipopt_options", """outlev = 0 expect_infeasible_problem = yes bound_push = 0.01 bound_frac = 0.01 warm_start_init_point = yes mu_strategy = adaptive mu_oracle = loqo max_iter=800 """)   #max_iter = 1000 mu_strategy = adaptive mu_oracle = loqo max_soc = 4
        ampl.option["presolve_eps"] = "6.82e-14"
        ampl.option['presolve'] = 1

        def format_indian_number(num):
            num_str = str(num)
            if len(num_str) <= 3:
                return num_str
            else:
                # Split the number into the last three digits and the rest
                last_three = num_str[-3:]
                remaining = num_str[:-3]
                # Add commas every two digits in the remaining part
                remaining = ','.join([remaining[max(i - 2, 0):i] for i in range(len(remaining), 0, -2)][::-1])
                return remaining + ',' + last_three
                
        
        # Solve the model
        # print(stop_flag.is_set())
        if not stop_flag.is_set():
            
            with contextlib.redirect_stdout(None):
                ampl.solve()
            
            print(f"Arc: {(u,v)}",
              f"Acyclic: True",
              f"Best_optimal: {format_indian_number(round(current_cost))}", 
              f"New_optimal: {format_indian_number(round(ampl.get_objective("total_cost").value()))}", 
              f"Solve_time: {round(ampl.get_value('_solve_elapsed_time'), 2)}", 
              f"Solve_result: {ampl.get_value("solve_result")}",
              f"Improved: {ampl.get_objective("total_cost").value() < current_cost}")
            
            
            
            # indegree_2_or_more = [node for node, indeg in network_graph.in_degree() if indeg >= 2]
            # super_source_out_arc = self.fix_arc_set()
            # all_duals = {}
            # for con_name, val in ampl.get_constraints():
            #     # Get dual values for each constraint
            #     dual_values = val.getValues()
            #     all_duals[con_name] = dual_values
    
            # inarc = []
            # for node in indegree_2_or_more:
            #     for (u, v) in network_graph.in_edges(node):
            #         if (u, v) in arcs:
            #             if (u,v) not in self.super_source_out_arc:
            #                 inarc.append((u,v))
            #         else:
            #             inarc.append((v,u))
                    

            # # Check if the stop flag is already set
            # if stop_flag.is_set():
            #     print(f"Worker for arc {arc} stopped as condition was met.")
            #     return {
            #         "total_cost": ampl.get_objective("total_cost").value(),
            #         "solve_result": ampl.get_value("solve_result"),
            #         "solve_time": round(ampl.get_value('_solve_elapsed_time'), 2),
            #         "improved": ampl.get_objective("total_cost").value() < current_cost,
            #         "acyclic_flow": network_graph.copy(), 
            #         "l": l,
            #         "q": q,
            #         "h": h,
            #     }
            
            # Check solution status
            if ampl.get_value("solve_result") == "solved" or ampl.get_value("solve_result") == "solved?":
                l = ampl.getVariable("l").getValues().to_dict()
                q = ampl.getVariable("q").getValues().to_dict()
                h = ampl.getVariable("h").getValues().to_dict()
                
                if ampl.get_objective("total_cost").value()<current_cost:
                    print(f"Condition met for arc {arc}, stopping parallel run.")
                    stop_flag.set()  # Set the stop flag
    
                return {
                    "arc":(u,v),
                    "total_cost": ampl.get_objective("total_cost").value(),
                    "solve_result": ampl.get_value("solve_result"),
                    "solve_time": round(ampl.get_value('_solve_elapsed_time'), 3),
                    "improved": ampl.get_objective("total_cost").value() < current_cost,
                    "acyclic_flow": network_graph.copy(), 
                    "l": l,
                    "q": q,
                    "h": h,
                    "nlp_run":True
                }
            else:
                return {
                    "arc":(u,v),
                    "total_cost": ampl.get_objective("total_cost").value(),
                    "solve_result": ampl.get_value("solve_result"),
                    "solve_time": round(ampl.get_value('_solve_elapsed_time'), 3),
                    "improved": ampl.get_objective("total_cost").value() < current_cost,
                    "acyclic_flow": network_graph.copy(), 
                    "l": l,
                    "q": q,
                    "h": h,
                    "nlp_run":True
                }
        return {
                "arc":(u,v),
                "total_cost": 'inf',
                "solve_result": "not_solved",
                "solve_time": 0,
                "improved": False,
                "acyclic_flow": None, 
                "l": None,
                "q": None,
                "h": None,
                "nlp_run":False
            }
                
    def run_parallel(self, network_graph, ampl_config):
        best_result = None
        improved = False
        
        
        current_cost =  ampl_config["current_cost"]
        best_acyclic_flow =  ampl_config["best_acyclic_flow"]
        l =  ampl_config["l"]
        q =  ampl_config["q"]
        h =  ampl_config["h"]
        
        # t =  ampl_config["t"]
        # all_duals =  ampl_config["all_duals"]
        # inarc =  ampl_config["inarc"]
        
        # Prepare the data for each worker
        # acyclic_arc_set = list(network_graph.edges)
        # data = [(arc, network_graph.copy(), ampl_config) for arc in self.acyclic_arc_set]
        
        # global stop_flag, pool
        
        # Initialize the shared stop flag
        # stop_flag = multiprocessing.Value('i', 0)  # Shared flag to stop workers
        
        # def check_result(result):
        #     """
        #     Callback function to check the worker's result.
        #     If the condition is met, set the stop flag to True.
        #     """
        #     global stop_flag
        #     if result["total_cost"] < current_cost:
        #         stop_flag.value = True  # Stop further processing
        #         print(f"Condition met. Stopping parallel computation. Total cost: {result['total_cost']}")
        #         pool.terminate()  # Terminate the pool immediately
        
        data = [(arc, network_graph.copy(),current_cost, best_acyclic_flow, self.data_number, l, q, h ) for arc in self.sorted_delta_arc]
        
        # print(data)
        # Run workers in parallel
        # with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        #     results = pool.map(self.worker, data)
        
        # nproc = os.cpu_count()
        # print(nproc, "processors available")
        # print()
        # pool = multiprocessing.Pool(8)
        # results = pool.map(self.worker, data)
        # Close the pool and wait for all tasks to complete
        # pool.close()
        # pool.join()
        
        # try:
        #     results = pool.starmap(self.worker, [(d, stop_flag) for d in data] )
        # finally:
        #     pool.close()
        #     pool.join()
        
        # Use a Manager to create a shared stop_flag
        # with multiprocessing.Manager() as manager:
        #     stop_flag = manager.Event()  # Shared event object
    
        #     # Create the pool of workers
        #     with multiprocessing.Pool(processes=8) as pool:
        #         try:
        #             results = pool.starmap(self.worker, [(d, stop_flag) for d in data])
        #         finally:
        #             pool.close()
        #             pool.join()
    
        # with multiprocessing.Manager() as manager:
        #     stop_flag = manager.Event()  # Shared event object
    
        #     with multiprocessing.Pool(processes=4) as pool:
        #         results = []
    
        #         # Use `imap_unordered()` instead of `starmap()` starmap_async
        #         for result in pool.starmap_async(self.worker, [(d, stop_flag) for d in data]):
        #             if stop_flag.is_set():  
        #                 pool.terminate()  # Kill all remaining processes
        #                 break  # Exit the loop
                
        #         pool.close()
        #         pool.join()
    
        #         print("Parallel execution stopped.")

        with multiprocessing.Manager() as manager:
            stop_flag = manager.Event()  # Shared event object

            with multiprocessing.Pool(processes=8) as pool:
                try:
                    # results = pool.starmap_async(self.worker, [(d, stop_flag) for d in data])  #apply_async, starmap_async starmap
                    results = [pool.apply_async(self.worker, args=(d, stop_flag)) for d in data]
                    # results = pool.starmap(self.worker, [(d, stop_flag) for d in data])
                    
                    # Monitor if stop_flag is set
                    # while not results.ready():
                    #     if stop_flag.is_set():
                    #         pool.terminate()  # Stop all processes
                    #         break
                    #     time.sleep(0.5)  # Avoid busy-waiting
                    # results = results.get()
                    
                    while not all(res.ready() for res in results):
                        if stop_flag.is_set():
                            pool.terminate()
                            break
                        time.sleep(0.5)
                    
                    results = [res.get() for res in results if res.ready()]  # Retrieve only ready results
                    
                    pool.close()
                    pool.join()

                    print("Parallel execution stopped.")
                
                except Exception as e:
                    print(f"Error: {e}")
                    pool.terminate()
                    pool.join()
        
        visited_nodes = [r["arc"][1] for r in results if r["solve_result"]!="not_solved"]
        self.visited_nodes.extend(visited_nodes)
        print("visited_nodes:",self.visited_nodes)

        # visited_arcs = [r["arc"] for r in results if r["solve_result"] != "not_solved"]
        # self.visited_arcs.extend(visited_arcs)
        
        self.number_of_nlp += len(visited_nodes)
        # Filter valid results
        valid_results = [r for r in results if r["solve_result"] == "solved" or r["solve_result"] == "solved?"]
        # nlp_list = [r for r in results if r["nlp_run"] == True]
        # self.number_of_nlp += len(nlp_list)
        
        # solve_time = 0
        # for i in range(len(nlp_list)):
        #     solve_time += nlp_list[i]["solve_time"] 
        
        # self.solver_time +=  solve_time
        
        if valid_results:
            # Find the best result
            best_result = min(valid_results, key=lambda r: r["total_cost"])
            
            # print(best_result)
            
            if best_result["total_cost"] < ampl_config["current_cost"]:
                ampl_config["current_cost"] = best_result["total_cost"]
                ampl_config["best_acyclic_flow"] = best_result["acyclic_flow"]
                ampl_config["l"] = best_result["l"]
                ampl_config["q"] = best_result["q"]
                ampl_config["h"] = best_result["h"]
                # ampl_config["t"] = best_result["t"]
                # ampl_config["all_duals"]: best_result["all_duals"]
                # ampl_config["inarc"]: best_result["inarc"]
                improved = True
                ampl_config["best_acyclic_flow"] = best_result["acyclic_flow"].copy()
                network_graph = best_result["acyclic_flow"].copy()
                arc = best_result["arc"]
                print("arc", arc)
                print(f"New best solution found! Total cost: {ampl_config['current_cost']}", f"Time_count: {round(time.time() - self.start_time, 2)}")
            else:
                improved = False
                arc = best_result["arc"]
                # print("arc", arc)
                print("No improved solutions found in this iteration.")
        else:
            improved = False
            arc = None
            # print("arc", arc)
            print("No improved solutions found in this iteration.")
        # else:
        #     print("No improved solutions found in this iteration.")

        # # Stop if no improvement
        # if not improved:
        #     break

        # Update the graph for the next iteration
        
        return network_graph, ampl_config, improved, arc

    def iterate_acyclic_flows(self):
        """Iterate to find improved acyclic flows by attempting arc reversals while maintaining acyclicity."""
        improved = True 
        
        self.best_acyclic_flow = self.network_graph.copy()
        
        # self.indegree_2_or_more = [node for node, indeg in self.best_acyclic_flow.in_degree() if indeg >= 2]
        # print("indegree_2_or_more:", self.indegree_2_or_more)
        
        if self.solve_result == "solved":
            current_cost = self.total_cost
            self.l = self.ampl.getVariable('l').getValues().to_dict()
            self.q = self.ampl.getVariable('q').getValues().to_dict()
            self.h = self.ampl.getVariable('h').getValues().to_dict()
            self.t = self.ampl.getVariable('t').getValues().to_dict()
            
            self.all_duals = {}
            for con_name, val in self.ampl.get_constraints():
                # Get dual values for each constraint
                dual_values = val.getValues()
                self.all_duals[con_name] = dual_values
        
        elif self.solve_result != "solved":
            current_cost = 10e+14
            self.l = None
            self.q = None
            self.h = None
        
        iteration = 1
        best_arc = None
        
        self.inarc = []
        for node in self.indegree_2_or_more:
            for (u, v) in self.best_acyclic_flow.in_edges(node):
                if (u, v) in self.arcs:
                    if (u,v) not in self.super_source_out_arc:
                        self.inarc.append((u,v))
                else:
                    self.inarc.append((v,u))
        
        # self.plot_graph(self.super_source_out_arc, current_cost, 0, self.q, self.h, self.D, (0,0), self.l, self.C)
        self.network_graph = self.best_acyclic_flow.copy()
        self.acyclic_arc_set = self.acyclic_arcs()
        sum_arc = {}       
        delta_arc = {}
        for node in self.indegree_2_or_more:
            sum_arc[node]=0
            for (u,v) in self.best_acyclic_flow.in_edges(node):
                if (u,v) in self.arcs:
                    sum_arc[node] += abs(self.q[u,v])
                else:
                    sum_arc[node] += abs(self.q[v,u])
            
            for (u,v) in self.best_acyclic_flow.in_edges(node):
                # delta_arc[u,v] = 0
                if (u,v) in self.arcs:
                    if (u,v) in self.acyclic_arc_set:
                        # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[u,v]))
                        delta_arc[u,v] = self.D[node] - sum_arc[node] + abs(self.q[u,v])
                else:
                    if (u,v) in self.acyclic_arc_set:
                        # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[v,u]))
                        delta_arc[u, v] = self.D[node] - sum_arc[node] + abs(self.q[v,u])
                    
        sorted_dict = dict(sorted(delta_arc.items(), key=lambda item: abs(item[1]), reverse=True))

        
        self.sorted_nodes = sorted(self.indegree_2_or_more, key=lambda node: self.D[node], reverse=True)
        print("sorted_nodes:",self.sorted_nodes)
        self.visited_nodes = []
        # self.visited_arcs = []
        # if self.visited_nodes:
        #     self.sorted_nodes = [item for item in self.sorted_nodes if item not in self.visited_nodes]
        print("sorted_nodes", self.sorted_nodes)
        final_sorted_arcs = []
        
        for node in   self.sorted_nodes:
            node_arcs = [(u, v) for (u, v) in self.best_acyclic_flow.in_edges(node) if (u, v) in delta_arc]
            # node_arcs_sorted = sorted(node_arcs, key=lambda arc: abs(delta_arc[arc]), reverse=True)
            # final_sorted_arcs.extend(node_arcs_sorted)
            if node_arcs:  # Ensure there are incoming arcs before picking the max
                max_arc = max(node_arcs, key=lambda arc: abs(delta_arc[arc]))  # Pick the arc with max absolute delta_arc
                final_sorted_arcs.append(max_arc)
                # if self.best_acyclic_flow.in_degree(node) >=3:
                #     min_arc = min(node_arcs, key=lambda arc: abs(delta_arc[arc]))  # Pick the arc with max absolute delta_arc
                #     final_sorted_arcs.append(min_arc)
                
                
        # Store final sorted arc order
        self.sorted_delta_arc = final_sorted_arcs
        # print(sorted_dict)
        # self.sorted_delta_arc = list(sorted_dict.keys())
        # print("sorted_delta_arc:",self.sorted_delta_arc)
        # ampl_config = {
        #     "current_cost": current_cost,
        #     "best_acyclic_flow": self.network_graph,
        #      "l": self.l,
        #      "q": self.q,
        #      "h": self.h,
        #      "t": self.t,
        #      "all_duals": self.all_duals,
        #     "inarc": self.inarc
        # }
        
        ampl_config = {
            "current_cost": current_cost,
            "best_acyclic_flow": self.network_graph,
            "l": self.l,
            "q": self.q,
            "h": self.h,
        }
        
        while improved and self.sorted_delta_arc:
            print("\n******************************************************************************************************************************************")
            print("Iteration :",iteration, "\n")
            # print(self.sorted_delta_arc)
            # print("sorted_delta_arc:",self.sorted_delta_arc)
            # self.best_acyclic_flow, improved, current_cost, self.l, self.q, self.h, best_arc = self.iterate_arc(iteration, improved, current_cost, best_arc)
            self.network_graph, ampl_config, improved, arc = self.run_parallel(self.network_graph, ampl_config)
            self.best_acyclic_flow = self.network_graph
            self.acyclic_arc_set = self.acyclic_arcs()
            # print("acyclic_arc_set:",acyclic_arc_set)
            sum_arc = {}       
            delta_arc = {}
            self.indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
            # print("self.indegree_2_or_more", self.indegree_2_or_more)
            for node in self.indegree_2_or_more:
                sum_arc[node]=0
                for (u,v) in self.best_acyclic_flow.in_edges(node):
                    if (u,v) in self.arcs:
                        sum_arc[node] += abs(self.q[u,v])
                    else:
                        sum_arc[node] += abs(self.q[v,u])
                
                for (u,v) in self.best_acyclic_flow.in_edges(node):
                    # delta_arc[u,v] = 0
                    if (u,v) in self.arcs:
                        if (u,v) in self.acyclic_arc_set:
                            # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[u,v]))
                            delta_arc[u,v] = self.D[node] - sum_arc[node] + abs(self.q[u,v])
                    else:
                        if (u,v) in self.acyclic_arc_set:
                            # print(f"Delta{u}_{v}:", self.D[node] - sum_arc[node] + abs(self.q[v,u]))
                            delta_arc[u, v] = self.D[node] - sum_arc[node] + abs(self.q[v,u])
                        

            self.sorted_nodes = sorted(self.indegree_2_or_more, key=lambda node: self.D[node], reverse=True)
            # self.visited_nodes = []
            if self.visited_nodes:
                self.sorted_nodes = [item for item in self.sorted_nodes if item not in self.visited_nodes]
            print("\nsorted_nodes", self.sorted_nodes, "\n")
            final_sorted_arcs = []
            
            for node in   self.sorted_nodes:
                node_arcs = [(u, v) for (u, v) in self.best_acyclic_flow.in_edges(node) if (u, v) in delta_arc]
                # node_arcs_sorted = sorted(node_arcs, key=lambda arc: abs(delta_arc[arc]), reverse=True)
                # final_sorted_arcs.extend(node_arcs_sorted)
                if node_arcs:  # Ensure there are incoming arcs before picking the max
                    max_arc = max(node_arcs, key=lambda arc: abs(delta_arc[arc]))  # Pick the arc with max absolute delta_arc
                    final_sorted_arcs.append(max_arc)
            # print(sorted_dict)
            self.sorted_delta_arc = final_sorted_arcs
            
            # if improved:
            #     (i,j) = arc
            #     self.sorted_delta_arc.remove((i,j))
            
            # sorted_dict = dict(sorted(delta_arc.items(), key=lambda item: abs(item[1]), reverse=True))
            # self.sorted_delta_arc = list(sorted_dict.keys())
            # if self.visited_arcs:
            #     self.sorted_delta_arc = [item for item in self.sorted_delta_arc if item not in self.visited_arcs]
            # print("\nsorted_nodes", self.sorted_nodes, "\n")
            
            # print("sorted_delta_arc:",self.sorted_delta_arc, "\n")
            # print(self.acyclic_arc_set)
            
            # print("Current best acyclic network:")
            # plt.figure(figsize=(10, 8))
            # nx.draw_spectral(best_acyclic_flow, with_labels=True)
            # plt.show()
            # super_source_out_arc = self.fix_arc()
            # super_source_out_arc.append(best_arc)
            iteration += 1
            # print(f"Current best solution: {current_cost}")
            # print(" ")
        
        print("\n*********************************************************Final best results***************************************************************")
        print("Water Network:", self.data_number)
        print(f"Final best objective: {ampl_config["current_cost"]}")

        print("Number of nlp problem solved:", self.number_of_nlp)
        print("Total number of iteration to solve the problem:", iteration-1)
        # print("length of the arcs: ", l, "\n")
        # print("flow in the arcs: ", q, "\n")
        # print("head value at node: ", h, "\n")
        # self.network_graph = best_acyclic_flow
        # self.load_model()
        # self.update_model()
        # self.multi_solve(current_cost)
        # self.ampl.close()
    
    # Function to suppress output
    @contextlib.contextmanager
    def suppress_output(self):
        # Open devnull to suppress the output
        with open(os.devnull, 'w') as devnull:
            # Redirect stdout and stderr
            old_stdout = sys.stdout
            old_stderr = sys.stderr
            sys.stdout = devnull
            sys.stderr = devnull
            try:
                yield
            finally:
                # Restore original stdout and stderr
                sys.stdout = old_stdout
                sys.stderr = old_stderr
    
    def solve(self):
        with self.suppress_output():
            """Solve the optimization problem."""
            self.ampl.option["solver"] = "ipopt"
            self.ampl.set_option("ipopt_options", "outlev = 0 expect_infeasible_problem = yes bound_push = 0.001 bound_frac = 0.001 warm_start_init_point = yes")   #max_iter = 1000
            # self.ampl.set_option("ipopt_options", """outlev = 0 expect_infeasible_problem = yes bound_push = 0.01 bound_frac = 0.01 warm_start_init_point = yes mu_strategy = adaptive mu_oracle = loqo  """)   #max_iter = 1000 mu_strategy = adaptive mu_oracle = loqo max_soc = 4 halt_on_ampl_error = yes
            self.ampl.option["presolve_eps"] = "6.82e-14"
            self.ampl.option['presolve'] = 1
            # self.ampl.option['solver_msg'] = 0
            # self.ampl.option['show_stats'] = 0
            self.ampl.solve()
            self.solve_result = self.ampl.solve_result
            self.total_cost = self.ampl.get_objective("total_cost").value()
        # print("Objective:", self.total_cost)
        # print("solve_result: ",self.solve_result)
        solve_time = self.ampl.get_value('_solve_elapsed_time')
        self.solver_time += solve_time
        self.number_of_nlp += 1
        
    def run(self):
        """Main method to run the optimization process."""
        
        self.start_time = time.time()
        print("Solve the original nonconvex optimization problem using IPOPT ")
        self.load_model()
        self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}l[i,j,k]*C[k];")
        # self.ampl.eval("minimize total_cost : sum{(i,j) in arcs} sum{k in pipes}1.2654*l[i,j,k]*(d[k])^1.327;")
        self.fix_leaf_arc_flow()
        
        self.super_source_out_arc = self.fix_arc_set()
        
        # self.ampl.eval("subject to con7{(i,j) in arcs}: -sum{k in nodes diff Source} D[k] <= q[i,j];")
        # self.ampl.eval("subject to con8{(i,j) in arcs}: q[i,j] <= sum{k in nodes diff Source} D[k];")
       
        # self.ampl.eval("display option ipopt_options;")     # Display all options in AMPL

        self.solve()
        
        print("Objective: ",self.total_cost)
        print("Solve_result: ",self.solve_result)
        print("Solve_time:", self.ampl.get_value('_solve_elapsed_time'),"\n")
        
        self.l = self.ampl.getVariable('l').getValues().to_dict()
        self.q = self.ampl.getVariable('q').getValues().to_dict()
        self.h = self.ampl.getVariable('h').getValues().to_dict()
        
        self.super_source_out_arc = self.fix_arc_set()
        self.network_graph = self.generate_random_acyclic_from_solution(self.q)
        
        # print("Fix the flow direction in optimization model and solve the updated model")
        
        self.inarc = []
        self.indegree_2_or_more = [node for node, indeg in self.network_graph.in_degree() if indeg >= 2]
        
        for node in self.indegree_2_or_more:
            for (u, v) in list(self.network_graph.in_edges(node)):
                if (u, v) in self.arcs:
                    self.inarc.append((u,v))
                else:
                    self.inarc.append((v,u))
        # print("inarc:", self.inarc)
        inarc_ = self.inarc
        
        inarc_set = []
        for (i, j) in self.inarc:
            if (i, j) in self.arcs:
                inarc_set.append(f"({i},{j})")
            else:
                inarc_set.append(f"({j},{i})")
        inarc_set = ", ".join(inarc_set)
        
        # self.display_results()
        self.iterate_acyclic_flows()
        elapsed_time = time.time() - self.start_time
        
        print("Solver_time:",self.solver_time, "seconds")
        print(f"Heuristic elapsed time:, {elapsed_time} seconds = {elapsed_time/60:.2f} minutes.")
        
if __name__ == "__main__":
    data_list = [
        "d1_Sample_input_cycle_twoloop",
        "d2_Sample_input_cycle_hanoi",
        "d3_Sample_input_double_hanoi",
        "d4_Sample_input_triple_hanoi",
        "d5_Taichung_input",
        "d6_HG_SP_1_4",
        "d7_HG_SP_2_3",
        "d8_HG_SP_3_4",
        "d9_HG_SP_4_2",
        "d10_HG_SP_5_5",
        "d11_HG_SP_6_3",
        "d12",
        "d13",
        "d14_NewYork",
        "d15_foss_poly_0",
        "d16_foss_iron",
        "d17_foss_poly_1",
        "d18_pescara",
        "d19_modena"
    ]
    
    # Select the data number here (0 to 18)
    data_number = 16
    input_data_file = f"/home/nitishdumoliya/waterNetwork/data/{data_list[data_number]}.dat"
    print("Water Network:", data_list[data_number],"\n")
    optimizer = WaterNetworkOptimizer("../m1Basic.mod", input_data_file, data_number, verbose=True)
    # optimizer = WaterNetworkOptimizer(sys.argv[1], sys.argv[3], sys.argv[2])
    optimizer.run()

Water Network: d5_Taichung_input 

Solve the original nonconvex optimization problem using IPOPT 
Objective:  8856873.473035004
Solve_result:  solved
Solve_time: 0.5078509999999999 

sorted_nodes: [13, 14, 1, 3, 4, 12, 15, 11]
sorted_nodes [13, 14, 1, 3, 4, 12, 15, 11]

******************************************************************************************************************************************
Iteration : 1 

Arc: (11, 15) Acyclic: True Best_optimal: 88,56,873 New_optimal: 89,73,285 Solve_time: 0.09 Solve_result: solved Improved: False
Arc: (6, 1) Acyclic: True Best_optimal: 88,56,873 New_optimal: 88,38,548 Solve_time: 0.09 Solve_result: solved Improved: True
Arc: (19, 14)Condition met for arc (6, 1), stopping parallel run. 
Acyclic: True Best_optimal: 88,56,873 New_optimal: 89,52,646 Solve_time: 0.24 Solve_result: solved Improved: False
Arc: (8, 13) Acyclic: True Best_optimal: 88,56,873 New_optimal: 90,59,877 Solve_time: 0.27 Solve_result: solved Improved: False
Arc: (12,